In [ ]:
from importlib import reload
# import os
import pandas as pd
import numpy as np
# import random as rd
import itertools
from datetime import datetime as dt

# from pathlib import Path
from sqlalchemy.orm import sessionmaker
import matplotlib.pyplot as plt
# from pprint import pprint
import seaborn as sns
# import statsmodels.api as sma
from imblearn.under_sampling import RandomUnderSampler
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit
import lightgbm as lgb
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import joblib

import sqlite_io as sio
import add_indicators as indic
import split_merge as sm
import balance
import lgbm_mngr
# import model_mngr as modmgr

# reload(modmgr)
reload(balance)
reload(lgbm_mngr)

<module 'balance' from 'G:\\Python\\MarketDataEnrichment\\dataset_mngr\\balance.py'>

In [2]:
# PATH_DATA = "C:\\Projets\\Data"
PATH_DATA = "G:\\Python\\Data"
PATH_DB_FWK=PATH_DATA+"\\sqlite\\dataset_market.db"
PATH_DB_STOCK=PATH_DATA+"\\sqlite\\dataset_paris_stock_adjusted.db"
PATH_DATA_DTS=PATH_DATA+"\\DTS_FULL\\"


CONNECTION TO SQLITE

In [3]:
if "con_stock" in locals():
        sio.close_connection(con_stock)
con_stock = sio.get_connection(str_db_path=PATH_DB_STOCK)

if "con_fwk" in locals():
        sio.close_connection(con_fwk)
con_fwk = sio.get_connection(str_db_path=PATH_DB_FWK)

my_session_maker = sessionmaker(bind=con_fwk)
session=my_session_maker()

table_stock="DS_PARIS_1D_ADJ_CLEAN"

GET DATA

In [4]:
dts_name="PARIS_TREND_1D_20D_V3"
multi_symbol="PARIS_STOCK"

df_base=sio.get_candles_to_df(session=session,con=con_stock, target_table=table_stock,tradable=True)
print(df_base[0:3])


DEBUG: SELECT CODE,OPEN_DATETIME, OPEN,HIGH,LOW,CLOSE,VOLUME FROM DS_PARIS_1D_ADJ_CLEAN can WHERE can.TIMEFRAME=1440 AND can.SK_SYMBOL IN (SELECT SK_SYMBOL FROM SYMBOL WHERE TRADABLE=1)   
                         OPEN    HIGH     LOW   CLOSE  VOLUME
CODE    OPEN_DATETIME                                        
ABCA.PA 2000-01-03     1.0267  1.0267  1.0267  1.0267  3490.0
        2000-01-04     1.0215  1.0215  1.0215  1.0215  4250.0
        2000-01-05     1.0382  1.0382  1.0382  1.0382  5960.0


In [ ]:
df_work=pd.DataFrame()
for code_value in df_base.index.get_level_values('CODE').unique():
    sub_df=df_base[df_base.index.get_level_values('CODE') == code_value]
    df_work_tmp = indic.add_indicators_to_df(con=con_fwk, df_in=sub_df, dts_name=dts_name,symbol=multi_symbol)
    df_work = pd.concat([df_work, df_work_tmp])
    
df_work.sort_index(inplace=True)
df_work.info() 

In [6]:
# print(df_base.index.get_level_values('CODE').unique())
# df_work[10000:10010]
# pd.set_option('display.max_columns', None)
# print(df_work.describe())

df_work.round(5).to_csv(
    PATH_DATA_DTS+"PARIS_TREND_1D_20D_V3_BASE_c", sep=",")

 FROM HERE TO LOAD THE BASE FILE

In [3]:
dts_name="PARIS_TREND_1D_20D_V3"
# dts_name="PARIS_TREND_1D_50D_V1"
multi_symbol="PARIS_STOCK"

df_work=pd.read_csv(PATH_DATA_DTS+dts_name+"_BASE.zip",sep=",",index_col=["OPEN_DATETIME","CODE"],parse_dates=["OPEN_DATETIME"])
df_work.head(5)

,,OPEN,HIGH,LOW,CLOSE,VOLUME,sma20,pos_sma20,sma50,sma200,pos_sma50,...,perf_atr14_5d,perf_adx14_5d,perf_adx14_neg_5d,perf_adx14_pos_5d,perf_adx14_dif_5d,avg_vol50,pos_avg_vol50,perf_williamsr_14_5d,williamsr_14_perf_10d,trix12
OPEN_DATETIME,CODE,,,,,,,,,,,,,,,,,,,,,
2010-04-26,AB.PA,12.98,12.98,12.20,12.68,62866.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-27,AB.PA,12.74,12.83,12.61,12.70,22370.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-28,AB.PA,12.70,12.70,12.41,12.50,8211.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-29,AB.PA,12.60,12.65,12.46,12.64,4676.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-30,AB.PA,12.63,12.71,12.55,12.65,4470.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#  extract 100 samples from 1000th line of COD AI.PA and write in a file for analyse
df_work.loc[(df_work.index.get_level_values('CODE') == "AI.PA")].iloc[5500:5600].to_csv(
    PATH_DATA_DTS+dts_name+"_BASE_AI_SAMPLES.csv", sep=",")

In [4]:
# add the column lab_perf_5d with this formula $$CLOSE$$.pct_change(periods=5).shift(-5)
df_work["lab_perf_5d"]=df_work.groupby("CODE")["CLOSE"].pct_change(periods=5).shift(-5)

C:\Users\Benoit\AppData\Local\Temp\ipykernel_16596\4180702997.py:2: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_work["lab_perf_5d"]=df_work.groupby("CODE")["CLOSE"].pct_change(periods=5).shift(-5)


Check Dataframe

In [5]:
df_check=df_work[df_work['stdev20_1d'] > 100]
# df_check=df_check[df_check['ret_1d'] <= 2]
print(df_check.index.get_level_values('CODE').unique())
# print(df_check.info())
# df_check[df_check.index.get_level_values('CODE')=='AKW.PA']
# df_check=df_work[df_work.index.get_level_values('CODE')=='AI.PA']
# CATG
# mask = df_work['stdev20_1d'] > 100
# df_work.drop(df_work[mask].index, inplace=True)
# # print 10 first lines of df_check with code=AAA.PA 
# code="CBR.PA"#'CBE.PA', 'CNV.PA'
# print(df_check[df_check.index.get_level_values('CODE')==code][0:20])

# # print the graph of DGE.PA with candles mode
# df_work[df_work.index.get_level_values('CODE')==code].plot(y=["OPEN","HIGH","LOW","CLOSE"],kind='line',figsize=(15, 10))  
# plt.show()

# drop lines where CODE=CBE.PA
# df_work.drop(df_work[df_work.index.get_level_values('CODE') == "CBE.PA"].index, inplace=True)
# df_work.round(5).to_csv(
#     PATH_DATA_DTS+"PARIS_TREND_1D_20D_V3_BASE_d", sep=",")



Index(['CNV.PA'], dtype='object', name='CODE')


In [9]:
print(df_work.shape)
df_work = indic.drop_indicators_by_type(
    con=con_fwk, df_in=df_work, dts_name=dts_name, symbol=multi_symbol, ind_type=0)
print(df_work.shape)
list_label = indic.get_ind_list_by_type_for_dts(
    con=con_fwk, dts_name=dts_name, symbol_code=multi_symbol, ind_type=2)
# print(type(list_label))
print(list_label)

<class 'pandas.core.frame.DataFrame'>
           LABEL
0  lab_perf_250d
1   lab_perf_20d
2   lab_perf_50d
3   lab_perf_10d
4    lab_perf_5d


In [ ]:
#  prepare the dataframe for the indictor analysis
lab_split='lab_perf_50d'
df_work['CODE'] = df_work.index.get_level_values('CODE')
df_work=df_work.droplevel('CODE')
df_work.sort_index(inplace=True)

df_split=sm.split_df_by_label_strat(
        df_in=df_work, list_label=[lab_split], split_timeframe="M",random_split=False,split_strat=(80,10,10))

df_class = df_split['df_'+lab_split+'_train']
df_class.sort_index(inplace=True)

for label in list_label['LABEL']:
    df_class = balance.add_class_by_lab_nb_lines(
        df_in=df_class, str_label=label, nb_class=10, bool_replace_label=False)
    print(df_class[label+"_class"].value_counts())
# df_work.sort_index(inplace=True)
# df_class=balance.add_class_by_lab_nb_lines(df_in=df_work,str_label="lab_perf_20d",nb_class=5,bool_replace_label=False)
# df_class=balance.add_class_by_lab_nb_lines(df_in=df_class,str_label="lab_perf_50d",nb_class=5,bool_replace_label=False)
df_class.sort_index(inplace=True)
#  count lines per class in lab_perf_20d
# print(df_class['lab_perf_20d_class'].value_counts())
# print(df_class['lab_perf_50d_class'].value_counts())

In [ ]:
##### SAVE !!!!##############
df_class.round(5).to_csv(
    PATH_DATA_DTS+"PARIS_TREND_1D_20D_V3_CLASS_TRAIN", sep=",")

In [ ]:
# analysis of the datafarme with some indicators to find a filter
list_indic_test=['pos_sma20','pos_sma50','pos_sma200','pos_sma20_50','pos_sma50_200',
'pos_sma20_200','perf_sma200_10d','pos_ema20','pos_ema50','pos_ema200','pos_ema20_50',
'pos_ema50_200','pos_ema20_200','perf_ema200_10d','ret_5d','aroon14_dif','macd_dif',
'stdev20_1d','pos_stdev20_sma20','stoch14_dif','adx14_dif','trix12']

for label in list_label['LABEL']:#list_label['LABEL']:
    df_indic_test=df_class[label+"_class"].value_counts()
    df_indic_test.sort_index(inplace=True)
    # convert to dataframe
    df_indic_test=pd.DataFrame(df_indic_test)
    print(type(df_indic_test))
    # print(df_class[df_class['rsi14'] > 50][label+"_class"].value_counts().sort_index())
    df_indic_test['rsi14']=df_class[df_class['rsi14'] > 50][label+"_class"].value_counts().sort_index()
    df_indic_test['williamsr_14']=df_class[df_class['williamsr_14'] > -50][label+"_class"].value_counts().sort_index()
    for indic_name in list_indic_test:
        print(indic_name)
        df_indic_test[indic_name]=df_class[df_class[indic_name] > 0][label+'_class'].value_counts().sort_index()
    
    df_indic_test.round(5).to_csv(PATH_DATA_DTS+"PARIS_TREND_1D_20D_indic_test_"+label, sep=",")
        
# print(df_indic_test)

# print nb lines with pos_sma200>0 and pos_sma200<0
# print(df_class[df_class['pos_sma50_200'] > 0].shape)
# print(df_class[df_class['pos_sma50_200'] < 0].shape)

# # print nb linesper class with pos_sma200>0 and pos_sma200<0
# print(df_class[df_class['pos_sma50_200'] > 0]['lab_perf_20d_class'].value_counts().sort_index())
# print(df_class[df_class['pos_sma50_200'] < 0]['lab_perf_20d_class'].value_counts().sort_index())

# print(df_class[df_class['pos_sma50_200'] > 0]['lab_perf_50d_class'].value_counts().sort_index())
# print(df_class[df_class['pos_sma50_200'] < 0]['lab_perf_50d_class'].value_counts().sort_index())

In [12]:
#  drop label lab_perf_250d and lab_perf_10d
# df_work.drop(columns=['lab_perf_250d','lab_perf_10d'],inplace=True)
list_rem=["lab_perf_250d","lab_perf_125d","pos_ema5_20","pos_ema10_20","perf_ema20_10d"
,"perf_ema50_5d","pos_ema5_50","perf_ema20_5d","perf_ema10_5d","pos_ema10_50","pos_ema20_50"
,"perf_ema10_10d","perf_ema200_10d","pos_ema20_200","perf_ema50_10d","pos_ema5_10","pos_ema50"
,"perf_ema200_5d","pos_ema5_200","pos_ema10_200","pos_ema200","pos_ema20"]
# df_work.drop(columns=['lab_perf_250d'],inplace=True)

# TODO MAKE A FUNCTION FOR THIS
df_filtered = df_work.copy()
list_feat = df_filtered.columns.values.tolist()
print(df_filtered.shape)

for item in list_rem:
    if item in list_feat:
        df_filtered.drop(columns=[item],inplace=True)
print(df_filtered.shape)



(1299247, 138)
(1299247, 117)


In [13]:
#  filter the dataframe with pos_sma20_200>=0
df_filtered=df_filtered[df_filtered['pos_sma20_200']>=0]
df_filtered.sort_index(inplace=True)
df_filtered.round(5).to_csv(
    PATH_DATA_DTS+"PARIS_TREND_1D_20D_V4_filtered", sep=",")

Start here for new datasets

In [3]:
# Start here for new datasets
dts_name="PARIS_TREND_1D_20D_V4"
df_filtered=pd.read_csv(PATH_DATA_DTS+dts_name+"_filtered.zip",sep=",",index_col=["OPEN_DATETIME","CODE"],parse_dates=["OPEN_DATETIME"])
print(df_filtered.shape)

(716338, 117)


In [5]:
# print list of columns
print(df_filtered.columns.values.tolist())

['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 'pos_sma20', 'pos_sma50', 'pos_sma200', 'pos_sma50_200', 'pos_sma20_50', 'rsi14', 'sma5_rsi14', 'sma20_rsi14', 'pos_bb20_hi', 'pos_bb20_lo', 'ret_1d', 'ret_5d', 'pos_top20', 'pos_top50', 'pos_bot20', 'pos_bot50', 'aroon14_up', 'aroon14_down', 'aroon14_dif', 'macd_dif', 'pos_top_200', 'pos_bot_200', 'stdev20_1d', 'stdev20_sma5', 'pos_stdev20_sma5', 'stdev20_sma20', 'pos_stdev20_sma20', 'pos_rsi14_sma5', 'pos_rsi14_sma20', 'pos_rsi14_sma5_20', 'lab_perf_20d', 'lab_perf_50d', 'stoch14', 'stoch14_signal', 'stoch14_dif', 'adx14', 'adx14_neg', 'adx14_pos', 'adx14_dif', 'pos_avg_vol14', 'tr_atr14', 'pos_sma20_200', 'williamsr_14', 'perf_sma_50_5d', 'perf_sma_200_5d', 'pos_sma10', 'pos_sma5', 'perf_sma_20_5d', 'perf_sma_10_5d', 'lab_perf_10d', 'perf_sma5_5d', 'perf_pos_sma10_10d', 'perf_sma10_5d', 'perf_sma10_10d', 'perf_pos_sma20_10d', 'perf_sma20_5d', 'perf_sma20_10d', 'perf_pos_sma50_10d', 'perf_sma50_5d', 'perf_sma50_10d', 'perf_pos_sma200_10d', '

In [4]:
df_filtered['CODE'] = df_filtered.index.get_level_values('CODE')
df_filtered=df_filtered.droplevel('CODE')
df_filtered.sort_index(inplace=True)
df_filtered.head(5)

,OPEN,HIGH,LOW,CLOSE,VOLUME,pos_sma20,pos_sma50,pos_sma200,pos_sma50_200,pos_sma20_50,...,perf_adx14_5d,perf_adx14_neg_5d,perf_adx14_pos_5d,perf_adx14_dif_5d,pos_avg_vol50,perf_williamsr_14_5d,williamsr_14_perf_10d,trix12,lab_perf_5d,CODE
OPEN_DATETIME,,,,,,,,,,,,,,,,,,,,,
1991-10-09,0.703,0.703,0.703,0.703,1792.0,-0.03047,-0.02040,0.12800,0.15150,0.01038,...,0.03223,2.51985,-2.51986,-5.03971,0.15837,0.00000,0.00000,-0.07907,-0.03001,BOI.PA
1991-10-10,0.703,0.703,0.703,0.703,864.0,-0.02844,-0.02040,0.12678,0.15025,0.00827,...,0.12260,10.51980,-10.51981,-21.03961,0.07732,-31.91489,0.00000,-0.10139,-0.01508,BOI.PA
1991-10-11,0.703,0.703,0.703,0.703,17920.0,-0.02578,-0.02040,0.12556,0.14900,0.00552,...,0.11071,-0.00001,-0.00001,0.00000,1.57693,0.00000,-5.48523,-0.11993,-0.01508,BOI.PA
1991-10-14,0.703,0.703,0.703,0.703,13984.0,-0.02311,-0.02033,0.12434,0.14768,0.00284,...,0.10024,-0.00001,-0.00001,0.00000,1.20180,0.00000,-10.97046,-0.13459,-0.00754,BOI.PA
1991-10-15,0.703,0.703,0.703,0.703,2208.0,-0.02105,-0.02033,0.12316,0.14647,0.00074,...,0.09098,-0.00001,-0.00001,0.00000,0.18969,0.00000,-10.97046,-0.14550,-0.01508,BOI.PA


In [5]:
label = "lab_perf_20d"
# algo_studied = "XG_BOOST_CLASS"
dts_name="PARIS_TREND_1D_20D_V4"

# df_work_lab = indic.drop_indicators_not_selected(con=con_fwk, df_in=df_work, dts_name=dts_name, symbol=multi_symbol,label=lab_studied,algo=algo_studied)

df_split=sm.split_df_by_label_strat(
    df_in=df_filtered, list_label=[label], split_timeframe="M",random_split=False,split_strat=(80,10,10))
df_selected = df_split['df_'+label+'_train']
df_valid = df_split['df_'+label+'_valid']
df_confirm = df_split['df_'+label+'_confirm']
df_selected.sort_index(inplace=True)
df_valid.sort_index(inplace=True)
df_confirm.sort_index(inplace=True)

print(f"selected: {df_selected.shape=} valid: {df_valid.shape=} confirm: {df_confirm.shape=}")


selected: df_selected.shape=(486879, 118) valid: df_valid.shape=(102506, 118) confirm: df_confirm.shape=(120468, 118)


In [14]:
label = "lab_perf_5d"
df_class=balance.add_class_by_lab_nb_lines(df_in=df_selected,str_label=label,nb_class=10,bool_replace_label=False)
min_max_lab_by_class = df_class.groupby(label+'_class')[label].agg(['min', 'max'])
print(min_max_lab_by_class)

                       min      max
lab_perf_5d_class                  
0                 -0.78866 -0.03894
1                 -0.03893 -0.02209
2                 -0.02208 -0.01198
3                 -0.01197 -0.00437
4                 -0.00436  0.00110
5                  0.00111  0.00816
6                  0.00817  0.01667
7                  0.01668  0.02816
8                  0.02817  0.04797
9                  0.04798  2.64539


In [ ]:
label = "lab_perf_20d"
df_class=balance.add_class_by_lab_nb_lines(df_in=df_selected,str_label=label,nb_class=5,bool_replace_label=False)
df_class.sort_index(inplace=True)
# categ_5={0:[-1,-0.02209],1:[-0.02209,-0.00437],2:[-0.00437,0.00816],3:[0.00816,0.02816],4:[0.02816,5]}
# categ_10={0:[-1,-0.03071],1:[-0.03071,-0.00522],2:[-0.00522,0.01406],3:[0.01406,0.04314],4:[0.04314,5]}
categ_20={0:[-1,-0.04141],1:[-0.04141,-0.00522],2:[-0.00522,0.02439],3:[0.02439,0.06687],4:[0.06687,5]}
# categ_50={0:[-1,-0.05826],1:[-0.05826,0.00124],2:[0.00124,0.05301],3:[0.05301,0.12161],4:[0.12161,5]}
df_class_val=balance.add_lab_by_class(df_in=df_valid,str_label=label, categ=categ_20,bool_replace_label=False) # categ
df_class_val.sort_index(inplace=True)
df_class_conf=balance.add_lab_by_class(df_in=df_confirm,str_label=label, categ=categ_20,bool_replace_label=False) # categ
df_class_conf.sort_index(inplace=True)
print(df_class.loc[:, label].dropna().iloc[[0, -1]])
print(df_class_val.loc[:, label].dropna().iloc[[0, -1]])
print(df_class_conf.loc[:, label].dropna().iloc[[0, -1]])
# df_class_clean=df_class.drop(['OPEN','HIGH','LOW','CLOSE','VOLUME','lab_perf_125d','lab_perf_20d','lab_perf_50d'],axis=1)
data = df_class[label]
print(data.value_counts().sort_index())
data_val = df_class_val[label]
print(data_val.value_counts().sort_index())
data_conf = df_class_conf[label]
print(data_conf.value_counts().sort_index())
# min_max_lab_by_class = df_class.groupby(label+'_class')[label].agg(['min', 'max'])
# print(min_max_lab_by_class)

#                         min      max
# lab_perf_20d_class                  
# 0                  -0.87743 -0.05172
# 1                  -0.05171 -0.00868
# 2                  -0.00867  0.02272
# 3                   0.02273  0.07058
# 4                   0.07059  3.82176

In [6]:
df_selected.round(5).to_pickle(PATH_DATA+"\DTS_FULL\PARIS_TREND_1D_V4_TRAIN.pkl")
df_valid.round(5).to_pickle(PATH_DATA+"\DTS_FULL\PARIS_TREND_1D_V4_VAL.pkl")
df_confirm.round(5).to_pickle(PATH_DATA+"\DTS_FULL\PARIS_TREND_1D_V4_CONF.pkl")

In [7]:
list_feat = df_selected.columns.values.tolist()
# list_feat.remove(label)
# df_x_train, col_y_train = sm.split_df_x_y(
#     df_in=df_class, list_features=list_feat, str_label=label, drop_na=True)
# df_x_val, col_y_val = sm.split_df_x_y(
#     df_in=df_class_val, list_features=list_feat, str_label=label, drop_na=True)
# df_x_conf, col_y_conf = sm.split_df_x_y(
#     df_in=df_class_conf, list_features=list_feat, str_label=label, drop_na=True)

print(df_selected.shape)
print(list_feat)


(486879, 118)
['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 'pos_sma20', 'pos_sma50', 'pos_sma200', 'pos_sma50_200', 'pos_sma20_50', 'rsi14', 'sma5_rsi14', 'sma20_rsi14', 'pos_bb20_hi', 'pos_bb20_lo', 'ret_1d', 'ret_5d', 'pos_top20', 'pos_top50', 'pos_bot20', 'pos_bot50', 'aroon14_up', 'aroon14_down', 'aroon14_dif', 'macd_dif', 'pos_top_200', 'pos_bot_200', 'stdev20_1d', 'stdev20_sma5', 'pos_stdev20_sma5', 'stdev20_sma20', 'pos_stdev20_sma20', 'pos_rsi14_sma5', 'pos_rsi14_sma20', 'pos_rsi14_sma5_20', 'lab_perf_20d', 'lab_perf_50d', 'stoch14', 'stoch14_signal', 'stoch14_dif', 'adx14', 'adx14_neg', 'adx14_pos', 'adx14_dif', 'pos_avg_vol14', 'tr_atr14', 'pos_sma20_200', 'williamsr_14', 'perf_sma_50_5d', 'perf_sma_200_5d', 'pos_sma10', 'pos_sma5', 'perf_sma_20_5d', 'perf_sma_10_5d', 'lab_perf_10d', 'perf_sma5_5d', 'perf_pos_sma10_10d', 'perf_sma10_5d', 'perf_sma10_10d', 'perf_pos_sma20_10d', 'perf_sma20_5d', 'perf_sma20_10d', 'perf_pos_sma50_10d', 'perf_sma50_5d', 'perf_sma50_10d', 'perf_pos_

In [18]:
dts_name="PARIS_TREND_1D_V4"
multi_symbol="PARIS_STOCK"
label = "lab_perf_20d"

# df_class=pd.read_csv(PATH_DATA_DTS+dts_name+"_TRAIN.zip",sep=",",index_col=["OPEN_DATETIME"],parse_dates=["OPEN_DATETIME"])
# df_class.dropna(subset=[label], inplace=True)
# df_class.sort_index(inplace=True)

# df_class_val=pd.read_csv(PATH_DATA_DTS+dts_name+"_VAL.zip",sep=",",index_col=["OPEN_DATETIME"],parse_dates=["OPEN_DATETIME"])
# df_class_val.dropna(subset=[label], inplace=True)
# df_class_val.sort_index(inplace=True)

# df_class_conf=pd.read_csv(PATH_DATA_DTS+dts_name+"_CONF.zip",sep=",",index_col=["OPEN_DATETIME"],parse_dates=["OPEN_DATETIME"])
# df_class_conf.dropna(subset=[label], inplace=True)
# df_class_conf.sort_index(inplace=True)
df_class=pd.read_pickle(PATH_DATA_DTS+dts_name+"_TRAIN.pkl")
df_class_val=pd.read_pickle(PATH_DATA_DTS+dts_name+"_VAL.pkl")
df_class_conf=pd.read_pickle(PATH_DATA_DTS+dts_name+"_CONF.pkl")

print(df_class.shape)
print(df_class_val.shape)
print(df_class_conf.shape)



(486879, 118)
(102506, 118)
(120468, 118)


In [19]:
categ_50_3={0:[-1,-0.01566],1:[-0.01566,0.07241],2:[0.07241,20]}
categ_50_5={0:[-1,-0.05826],1:[-0.05826,0.00124],2:[0.00124,0.05301],3:[0.05301,0.12161],4:[0.12161,15]}
categ_50_10={0:[-1,-0.11173],1:[-0.11173,-0.05826],2:[-0.05826,-0.02501],3:[-0.02501,0.00124],4:[0.00124,0.02632],
             5:[0.02632,0.05301],6:[0.05301,0.08305],7:[0.08305,0.12161],8:[0.12161,0.18600],9:[0.18600,20]}
list_categ_50_to_add={3:categ_50_3,5:categ_50_5,10:categ_50_10}

categ_20_3={0:[-1,-0.01542],1:[-0.01542,0.03633],2:[0.03633,20]}
categ_20_5={0:[-1,-0.04141],1:[-0.04141,-0.00522],2:[-0.00522,0.02439],3:[0.02439,0.06687],4:[0.06687,5]}
categ_20_10={0:[-1,-0.07407],1:[-0.07407,-0.04141],2:[-0.04141,-0.02099],3:[-0.02099,-0.00522],4:[-0.00522,0.00872],
             5:[0.00872,0.02439],6:[0.02439,0.04295],7:[0.04295,0.06687],8:[0.06687,0.10603],9:[0.10603,20]}
list_categ_20_to_add={3:categ_20_3,5:categ_20_5,10:categ_20_10}

categ_10_3={0:[-1,-0.01226],1:[-0.01226,0.02207],2:[0.02207,20]}
categ_10_5={0:[-1,-0.03071],1:[-0.03071,-0.00522],2:[-0.00522,0.01406],3:[0.01406,0.04314],4:[0.04314,5]}
categ_10_10={0:[-1,-0.05387],1:[-0.05387,-0.03071],2:[-0.03071,-0.01624],3:[-0.01624,-0.00522],4:[-0.00522,0.00367],
             5:[0.00367,0.01406],6:[0.01406,0.02657],7:[0.02657,0.04314],8:[0.04314,0.07055],9:[0.07055,20]}
list_categ_10_to_add={3:categ_10_3,5:categ_10_5,10:categ_10_10}

categ_5_3={0:[-1,-0.00925],1:[-0.00925,0.01359],2:[0.01359,20]}
categ_5_5={0:[-1,-0.02209],1:[-0.02209,-0.00437],2:[-0.00437,0.00816],3:[0.00816,0.02816],4:[0.02816,5]}
categ_5_10={0:[-1,-0.03894],1:[-0.03894,-0.02209],2:[-0.02209,-0.01198],3:[-0.01198,-0.00437],4:[-0.00437,0.00110],
             5:[0.00110,0.00816],6:[0.00816,0.01667],7:[0.01667,0.02816],8:[0.02816,0.04797],9:[0.04797,20]}
list_categ_5_to_add={3:categ_5_3,5:categ_5_5,10:categ_5_10}

list_label=['lab_perf_5d','lab_perf_10d','lab_perf_20d','lab_perf_50d']
dic_list_categ_to_add={'lab_perf_5d':list_categ_5_to_add,'lab_perf_10d':list_categ_10_to_add,
                       'lab_perf_20d':list_categ_20_to_add,'lab_perf_50d':list_categ_50_to_add}

for my_label in list_label:
    my_list_categ_to_add=dic_list_categ_to_add[my_label]
    for nb_class in my_list_categ_to_add.keys():
        suffix_added="_class_"+str(nb_class)
        lab_added=my_label+suffix_added
        print(f"{my_label=} {nb_class=} {lab_added=} {my_list_categ_to_add[nb_class]=}")
        df_class=balance.add_lab_by_class(df_in=df_class,str_label=my_label, categ=my_list_categ_to_add[nb_class],
                                        bool_replace_label=False,str_suffix_class=suffix_added) # categ
        df_class_val=balance.add_lab_by_class(df_in=df_class_val,str_label=my_label, categ=my_list_categ_to_add[nb_class],
                                            bool_replace_label=False,str_suffix_class=suffix_added) # categ
        df_class_conf=balance.add_lab_by_class(df_in=df_class_conf,str_label=my_label, categ=my_list_categ_to_add[nb_class],
                                            bool_replace_label=False,str_suffix_class=suffix_added) # categ

        df_class.sort_index(inplace=True)
        df_class_val.sort_index(inplace=True)
        df_class_conf.sort_index(inplace=True)

        print(df_class[lab_added].value_counts().sort_index()) # min 97366 
        print(df_class_val[lab_added].value_counts().sort_index()) # min 14774
        print(df_class_conf[lab_added].value_counts().sort_index()) # min 20566

# df_class=balance.add_lab_by_class(df_in=df_class,str_label=label, categ=categ_20_5,bool_replace_label=False) # categ
# df_class.sort_index(inplace=True)
# df_class_val=balance.add_lab_by_class(df_in=df_class_val,str_label=label, categ=categ_20_5,bool_replace_label=False) # categ
# df_class_val.sort_index(inplace=True)
# df_class_conf=balance.add_lab_by_class(df_in=df_class_conf,str_label=label, categ=categ_20_5,bool_replace_label=False) # categ
# df_class_conf.sort_index(inplace=True)



my_label='lab_perf_5d' nb_class=3 lab_added='lab_perf_5d_class_3' my_list_categ_to_add[nb_class]={0: [-1, -0.00925], 1: [-0.00925, 0.01359], 2: [0.01359, 20]}
lab_perf_5d_class_3
0.0    162253
1.0    162301
2.0    162325
Name: count, dtype: int64
lab_perf_5d_class_3
0.0    36657
1.0    34239
2.0    31610
Name: count, dtype: int64
lab_perf_5d_class_3
0.0    44562
1.0    34987
2.0    40919
Name: count, dtype: int64
my_label='lab_perf_5d' nb_class=5 lab_added='lab_perf_5d_class_5' my_list_categ_to_add[nb_class]={0: [-1, -0.02209], 1: [-0.02209, -0.00437], 2: [-0.00437, 0.00816], 3: [0.00816, 0.02816], 4: [0.02816, 5]}
lab_perf_5d_class_5
0.0    97348
1.0    97380
2.0    97382
3.0    97381
4.0    97388
Name: count, dtype: int64
lab_perf_5d_class_5
0.0    22829
1.0    20863
2.0    20187
3.0    20264
4.0    18363
Name: count, dtype: int64
lab_perf_5d_class_5
0.0    29037
1.0    22871
2.0    20153
3.0    22709
4.0    25698
Name: count, dtype: int64
my_label='lab_perf_5d' nb_class=10 lab_added

In [20]:
# save the 3 df in pkl format
df_class.to_pickle(PATH_DATA_DTS+dts_name+"_class_TRAIN.pkl")
df_class_val.to_pickle(PATH_DATA_DTS+dts_name+"_class_VAL.pkl")
df_class_conf.to_pickle(PATH_DATA_DTS+dts_name+"_class_CONF.pkl")

print(df_class.shape)
print(df_class_val.shape)
print(df_class_conf.shape)


(486879, 130)
(102506, 130)
(120468, 130)


START HERE !

In [6]:
dts_name="PARIS_TREND_1D_V4_class"
multi_symbol="PARIS_STOCK"
label = "lab_perf_20d"

# load the 3 pkl files in 3 df
df_class=pd.read_pickle(PATH_DATA_DTS+dts_name+"_TRAIN.pkl")
df_class_val=pd.read_pickle(PATH_DATA_DTS+dts_name+"_VAL.pkl")
df_class_conf=pd.read_pickle(PATH_DATA_DTS+dts_name+"_CONF.pkl")

print(f"{df_class.shape=} {df_class_val.shape=} {df_class_conf.shape=}")

df_class.shape=(486879, 130) df_class_val.shape=(102506, 130) df_class_conf.shape=(120468, 130)


In [ ]:
# clean V4 dataset to make a V5

new_dts_name="PARIS_TREND_1D_V5_class"

df_class_v5=df_class.copy()
df_class_val_v5=df_class_val.copy()
df_class_conf_v5=df_class_conf.copy()

#add column CODE to the index to get a multi index
df_class_v5=df_class_v5.set_index('CODE',append=True)
df_class_val_v5=df_class_val_v5.set_index('CODE',append=True)
df_class_conf_v5=df_class_conf_v5.set_index('CODE',append=True)

# drop col stdev20_sma20 and stdev20_sma5
df_class_v5.drop(columns=['stdev20_sma20','stdev20_sma5'],inplace=True)
df_class_val_v5.drop(columns=['stdev20_sma20','stdev20_sma5'],inplace=True)
df_class_conf_v5.drop(columns=['stdev20_sma20','stdev20_sma5'],inplace=True)

# delete ALVDM.PA before 2017
df_class_v5.drop(df_class_v5[(df_class_v5.index.get_level_values('CODE') == "ALVDM.PA") & 
                             (df_class_v5.index.get_level_values('OPEN_DATETIME') < "2017-01-01")].index,inplace=True)
df_class_val_v5.drop(df_class_val_v5[(df_class_val_v5.index.get_level_values('CODE') == "ALVDM.PA") &
                                    (df_class_val_v5.index.get_level_values('OPEN_DATETIME') < "2017-01-01")].index,inplace=True)
df_class_conf_v5.drop(df_class_conf_v5[(df_class_conf_v5.index.get_level_values('CODE') == "ALVDM.PA") &
                                      (df_class_conf_v5.index.get_level_values('OPEN_DATETIME') < "2017-01-01")].index,inplace=True)

# delete SAMS.PA LOUP.PA ELEC.PA CRSU.PA IMDA.PA LOCAL.PA EUR.PA ALFLE.PA BUR.PA
list_code_to_del=["SAMS.PA","LOUP.PA","ELEC.PA","CRSU.PA","IMDA.PA","LOCAL.PA","EUR.PA","ALFLE.PA","BUR.PA"]
for code in list_code_to_del:
    df_class_v5.drop(df_class_v5[(df_class_v5.index.get_level_values('CODE') == code)].index,inplace=True)
    df_class_val_v5.drop(df_class_val_v5[(df_class_val_v5.index.get_level_values('CODE') == code)].index,inplace=True)
    df_class_conf_v5.drop(df_class_conf_v5[(df_class_conf_v5.index.get_level_values('CODE') == code)].index,inplace=True)

# delete ALGIL.PA before 2012
df_class_v5.drop(df_class_v5[(df_class_v5.index.get_level_values('CODE') == "ALGIL.PA") & 
                             (df_class_v5.index.get_level_values('OPEN_DATETIME') < "2012-01-01")].index,inplace=True)
df_class_val_v5.drop(df_class_val_v5[(df_class_val_v5.index.get_level_values('CODE') == "ALGIL.PA") &
                                    (df_class_val_v5.index.get_level_values('OPEN_DATETIME') < "2012-01-01")].index,inplace=True)
df_class_conf_v5.drop(df_class_conf_v5[(df_class_conf_v5.index.get_level_values('CODE') == "ALGIL.PA") &
                                      (df_class_conf_v5.index.get_level_values('OPEN_DATETIME') < "2012-01-01")].index,inplace=True)

# delete EIFF.PA before 2007
df_class_v5.drop(df_class_v5[(df_class_v5.index.get_level_values('CODE') == "EIFF.PA") & 
                             (df_class_v5.index.get_level_values('OPEN_DATETIME') < "2007-01-01")].index,inplace=True)
df_class_val_v5.drop(df_class_val_v5[(df_class_val_v5.index.get_level_values('CODE') == "EIFF.PA") &
                                    (df_class_val_v5.index.get_level_values('OPEN_DATETIME') < "2007-01-01")].index,inplace=True)
df_class_conf_v5.drop(df_class_conf_v5[(df_class_conf_v5.index.get_level_values('CODE') == "EIFF.PA") &
                                      (df_class_conf_v5.index.get_level_values('OPEN_DATETIME') < "2007-01-01")].index,inplace=True)

# delete ALMDT.PA before 2020
df_class_v5.drop(df_class_v5[(df_class_v5.index.get_level_values('CODE') == "ALMDT.PA") & 
                             (df_class_v5.index.get_level_values('OPEN_DATETIME') < "2020-01-01")].index,inplace=True)
df_class_val_v5.drop(df_class_val_v5[(df_class_val_v5.index.get_level_values('CODE') == "ALMDT.PA") &
                                    (df_class_val_v5.index.get_level_values('OPEN_DATETIME') < "2020-01-01")].index,inplace=True)
df_class_conf_v5.drop(df_class_conf_v5[(df_class_conf_v5.index.get_level_values('CODE') == "ALMDT.PA") &
                                      (df_class_conf_v5.index.get_level_values('OPEN_DATETIME') < "2020-01-01")].index,inplace=True)

# delete GFC.PA before 1998
df_class_v5.drop(df_class_v5[(df_class_v5.index.get_level_values('CODE') == "GFC.PA") & 
                             (df_class_v5.index.get_level_values('OPEN_DATETIME') < "1998-01-01")].index,inplace=True)
df_class_val_v5.drop(df_class_val_v5[(df_class_val_v5.index.get_level_values('CODE') == "GFC.PA") &
                                    (df_class_val_v5.index.get_level_values('OPEN_DATETIME') < "1998-01-01")].index,inplace=True)
df_class_conf_v5.drop(df_class_conf_v5[(df_class_conf_v5.index.get_level_values('CODE') == "GFC.PA") &
                                      (df_class_conf_v5.index.get_level_values('OPEN_DATETIME') < "1998-01-01")].index,inplace=True)

print(f"{df_class_v5.shape=} {df_class_val_v5.shape=} {df_class_conf_v5.shape=}")

df_class_v5.to_pickle(PATH_DATA_DTS+new_dts_name+"_TRAIN.pkl")
df_class_val_v5.to_pickle(PATH_DATA_DTS+new_dts_name+"_VAL.pkl")
df_class_conf_v5.to_pickle(PATH_DATA_DTS+new_dts_name+"_CONF.pkl")



(455900, 127)
(98665, 127)
(117005, 127)


In [ ]:
dts_name="PARIS_TREND_1D_V5_class"
print(df_class_v5.head(5))
# print list of columns of df_class in a txt file
with open(PATH_DATA_DTS+dts_name+"_class_TRAIN_columns.txt", "w") as output:
    output.write(str(df_class.columns.values.tolist()))

In [10]:
df_class=df_class_v5.copy()
df_class_val=df_class_val_v5.copy()
df_class_conf=df_class_conf_v5.copy()

In [11]:
# ROUND THE DATASETS
# round_to_significant_digits(df : pd.DataFrame, digits : int, list_col_int:list=None, list_col_ignore:list=None)
pd.set_option('display.max_columns', None)

digits=3
list_col_int=["rsi14","sma5_rsi14","sma20_rsi14","aroon14_up","aroon14_down","aroon14_dif",
"stoch14","stoch14_signal","stoch14_dif","adx14","adx14_neg","adx14_pos",
"perf_rsi14_5d","perf_rsi14_10d","aroon14_up_lag_5d","aroon14_down_lag_5d",
"aroon14_dif_lag_5d",	"perf_stoch14_5d","perf_stoch14_signal_5d",
"perf_stoch14_dif_5d","perf_adx14_neg_5d",	"perf_adx14_pos_5d",	
"perf_williamsr_14_5d","williamsr_14_perf_10d","williamsr_14"	]

list_col_ignore=["CODE","OPEN", "HIGH", "LOW", "CLOSE", "VOLUME","lab_perf_20d","lab_perf_50d","lab_perf_125d",
                  "lab_perf_10d","lab_perf_5d", "lab_perf_5d_class_3",
 "lab_perf_5d_class_5", "lab_perf_5d_class_10", "lab_perf_10d_class_3", "lab_perf_10d_class_5",
 "lab_perf_10d_class_10", "lab_perf_20d_class_3", "lab_perf_20d_class_5", "lab_perf_20d_class_10",
 "lab_perf_50d_class_3", "lab_perf_50d_class_5", "lab_perf_50d_class_10"]

df_class_r=balance.round_to_significant_digits(df=df_class,digits=digits,
                                               list_col_int=list_col_int,list_col_ignore=list_col_ignore)
df_class_val_r=balance.round_to_significant_digits(df=df_class_val,digits=digits,
                                               list_col_int=list_col_int,list_col_ignore=list_col_ignore)
df_class_conf_r=balance.round_to_significant_digits(df=df_class_conf,digits=digits,
                                               list_col_int=list_col_int,list_col_ignore=list_col_ignore)


df_class_r.to_pickle(PATH_DATA_DTS+dts_name+"_R"+str(digits)+"_TRAIN.pkl")
df_class_val_r.to_pickle(PATH_DATA_DTS+dts_name+"_R"+str(digits)+"_VAL.pkl")
df_class_conf_r.to_pickle(PATH_DATA_DTS+dts_name+"_R"+str(digits)+"_CONF.pkl")


In [19]:
# read the 3 pkl files
dts_name="PARIS_TREND_1D_V5_class_R3"

df_class=pd.read_pickle(PATH_DATA_DTS+dts_name+"_TRAIN.pkl")
df_class_val=pd.read_pickle(PATH_DATA_DTS+dts_name+"_VAL.pkl")
df_class_conf=pd.read_pickle(PATH_DATA_DTS+dts_name+"_CONF.pkl")


In [ ]:
pd.set_option('display.max_columns', None)
print(f"{df_class.shape=} {df_class_val.shape=} ")
print(f"{df_class.describe()}")
print(f"{df_class[1000:1050]}")


In [20]:
# Clip of the features 
feature_limits = {
    'pos_sma20': [-0.5, 0.5],
    'pos_sma50': [-0.5, 1],
    'pos_sma200': [-0.5, 1],
    'pos_sma50_200': [-0.25, 1],
    'ret_1d': [-0.25, 0.25],
    'ret_5d': [-0.5, 0.5],
    'stdev20_1d': [None, 5],
    'pos_stdev20_sma20': [None, 4],
    'macd_dif': [-5, 5],
    'pos_stdev20_sma5': [None, 1],
    'pos_ema50_perf_5d': [-10, 10],
    'pos_ema50_perf_10d': [-10, 10],
    'pos_bb20_lo_lag_5d': [None, 2],
    'pos_bot20_lag_5d': [None, 1],
    'pos_bot50_lag_10d': [None, 1],
    'perf_atr14_5d': [None, 2],
    'perf_adx14_5d': [None, 2],
    'pos_avg_vol50': [None, 10],
    'pos_bot20': [None, 1],
    'pos_bot50': [None, 1],
    'pos_bot_200': [None, 4],
    'pos_rsi14_sma5': [-0.75, 0.75],
    'pos_rsi14_sma20': [-0.75, 0.75],
    'pos_rsi14_sma5_20': [-0.75, 0.75],
    'pos_sma20_200': [None, 1],
    'pos_sma10': [-0.5, 0.5],
    'pos_sma5': [-0.25, 0.25],
    'perf_pos_sma10_10d': [-0.5, 0.5],
    'perf_sma10_5d': [-0.25, 0.25],
    'perf_sma10_10d': [-0.5, 0.5],
    'perf_pos_sma20_10d': [-0.5, 0.5],
    'perf_sma20_10d': [-0.25, 0.25],
    'perf_pos_sma50_10d': [-0.5, 0.5],
    'perf_pos_sma200_10d': [-0.5, 0.5],
    'pos_sma5_50': [-0.5, 0.5],
    'pos_sma5_200': [-0.5, 1],
    'pos_sma10_50': [-0.5, 0.5],
    'pos_sma10_200': [-0.25, 1],
    'perf_ema5_5d': [-0.25, 0.25],
    'pos_ema10': [-0.25, 0.25],
    'perf_pos_ema10_5d': [-0.25, 0.25],
    'perf_pos_ema10_10d': [-0.25, 0.25],
    'perf_pos_ema20_5d': [-0.25, 0.25],
    'perf_pos_ema20_10d': [-0.5, 0.5],
    'perf_pos_ema200_5d': [-0.5, 0.5],
    'perf_pos_ema200_10d': [-0.5, 0.5],
    'pos_bb20_lo_lag_5d': [-0.5, 0.5],
    'ret_10d': [-0.5, 0.5],
    'ret_20d': [-0.5, 0.5],
    'pos_bot20_lag_5d': [None, 0.5],
    'pos_bot50_lag_10d': [None, 1],
    'macd_fast': [-5, 5],
    'macd_slow': [-5, 5],
    'perf_atr14_5d': [None, 2]
}

for feature, limits in feature_limits.items():
    min_val, max_val = limits
    try:
        df_class = balance.clipping_col(df_class, feature, min_val, max_val)
    except KeyError:
        print(f"Feature '{feature}' not found in df_class. Skipping...")
    try:
        df_class_val = balance.clipping_col(df_class_val, feature, min_val, max_val)
    except KeyError:
        print(f"Feature '{feature}' not found in df_class_val. Skipping...")
    try:
        df_class_conf = balance.clipping_col(df_class_conf, feature, min_val, max_val)
    except KeyError:
        print(f"Feature '{feature}' not found in df_class_conf. Skipping...")


df_class.to_pickle(PATH_DATA_DTS+dts_name+"_CLIP_TRAIN.pkl")
df_class_val.to_pickle(PATH_DATA_DTS+dts_name+"_CLIP_VAL.pkl")
df_class_conf.to_pickle(PATH_DATA_DTS+dts_name+"_CLIP_CONF.pkl")

In [ ]:
list_col=['pos_sma20','pos_sma50','pos_sma200','pos_sma50_200','ret_1d','ret_5d'
,'stdev20_1d','pos_stdev20_sma20','macd_dif','pos_stdev20_sma5'
,'pos_ema50_perf_5d','pos_ema50_perf_10d','pos_bb20_lo_lag_5d','pos_bot20_lag_5d','pos_bot50_lag_10d'
,'perf_atr14_5d','perf_adx14_5d','pos_avg_vol50','pos_bot20','pos_bot50','pos_bot_200'
,'pos_rsi14_sma5','pos_rsi14_sma20','pos_rsi14_sma5_20','pos_sma20_200','pos_sma10'
,'pos_sma5','perf_pos_sma10_10d','perf_sma10_5d','perf_sma10_10d'  ,'perf_pos_sma20_10d'
,'perf_sma20_10d'  ,'perf_pos_sma50_10d','perf_pos_sma200_10d','pos_sma5_50'   
,'pos_sma5_200','pos_sma10_50' ,'pos_sma10_200','perf_ema5_5d'      ,'pos_ema10'  
,'perf_pos_ema10_5d'  ,'perf_pos_ema10_10d','perf_pos_ema20_5d'  ,'perf_pos_ema20_10d'  
,'perf_pos_ema200_5d'  ,'perf_pos_ema200_10d'
,'pos_bb20_lo_lag_5d','ret_10d'        ,'ret_20d','pos_bot20_lag_5d'  ,'pos_bot50_lag_10d'
,'macd_fast'      ,'macd_slow','perf_atr14_5d']

# for each col in list_col print a graph of the distribution of the values
for col in list_col:
    print(col)
    sns.histplot(data=df_class, x=col, kde=True, bins=50, color='blue')
    plt.show()

In [ ]:
# print lines from df_class with stdev20_sma20>4 and open_datetime > 2018-01-01
df_check=df_class[(df_class['macd_fast'] >5) 
                   & (df_class.index.get_level_values('OPEN_DATETIME') > '2016-01-01')
                # & (df_class['CODE'] =='ABVX.PA')
                  ]

print(df_check.loc[:, ['CODE', 'macd_fast']].head(20))

# print(df_class['CODE'].unique().shape)

# print(df_check['CODE'].unique())

In [ ]:
# print the candle graph of BUR.PA from '2015-10-01' to '2016-02-01'
df_graph = df_class[(df_class['CODE'] == 'BUR.PA') & (df_class.index > '2015-10-01') & (df_class.index < '2016-02-01')]
df_graph.plot(y=["stdev20_1d","pos_stdev20_sma20"],kind='line',figsize=(15, 10))
plt.show()

# print(df_graph.loc[:, [ 'pos_stdev20_sma20']])

In [ ]:
df_class.drop(columns=list_rem,inplace=True,errors="ignore")

list_feat = df_class.columns.values.tolist()
# remove list_rem from list_feat, don't crash if not in list

print(f"{list_feat.__len__()=}")
list_feat = [x for x in list_feat if x not in list_rem]

print(f"{list_feat.__len__()=}")

df_x_train, col_y_train = sm.split_df_x_y(
    df_in=df_class,list_features=list_feat, str_label=label, drop_na=True)
# df_x_train.drop(list_rem, axis=1, inplace=True)
print(df_x_train.shape)

In [5]:
# print all columns starting by lab_perf
list_perf = [col for col in df_class.columns if col.startswith('lab_perf')]
print(list_perf)
# ['lab_perf_250d', 'lab_perf_20d', 'lab_perf_50d', 'lab_perf_10d', 'lab_perf_5d', 'lab_perf_20d_class']

['lab_perf_20d', 'lab_perf_50d', 'lab_perf_10d', 'lab_perf_5d']


In [6]:
# df_class=df_selected.copy()
# df_class_val=df_valid.copy()

list_perf=["OPEN", "HIGH", "LOW", "CLOSE", "VOLUME",
"lab_perf_50d","lab_perf_10d","lab_perf_125d","lab_perf_5d", "CODE"]

# remove columns from list_perf except label
df_class.drop(columns=list_perf,inplace=True,errors="ignore")
df_class_val.drop(columns=list_perf,inplace=True,errors="ignore")

# for class datasets 
# rename column  label+'_class' by label
# df_class.rename(columns={label+'_class':label},inplace=True)
# df_class_val.rename(columns={label+'_class':label},inplace=True)

# print(df_class[label].value_counts().sort_index())
# print(df_class_val[label].value_counts().sort_index())

# for regression datasets
print(f"{df_class.shape=} {df_class_val.shape=}")
print(f"{df_class[label].mean()=} {df_class[label].min()=} {df_class[label].max()=}")
print(f"{df_class_val[label].mean()=} {df_class_val[label].min()=} {df_class_val[label].max()=}")




df_class.shape=(486879, 109) df_class_val.shape=(102506, 109)
df_class[label].mean()=0.014540433187711938 df_class[label].min()=-0.80165 df_class[label].max()=2.4383
df_class_val[label].mean()=0.002089393986693463 df_class_val[label].min()=-0.76396 df_class_val[label].max()=10.14943


In [ ]:
#  for classification datasets !!!!

pd.set_option('display.max_columns', None)
# print(df_x_train.describe())

# list_feat=df_x_train.columns.values.tolist()
list_feat=None

# add_indicators.round_to_significant_digits(df_in: pd.DataFrame, digits: int = 4, list_min2:list=[]) -> pd.DataFrame: 
def round_to_significant_digits(df, digits):
    list_zero=["rsi14","sma5_rsi14","sma20_rsi14","aroon14_up","aroon14_down","aroon14_dif",
"stoch14","stoch14_signal","stoch14_dif","adx14","adx14_neg","adx14_pos",
"perf_rsi14_5d","perf_rsi14_10d","aroon14_up_lag_5d","aroon14_down_lag_5d",
"aroon14_dif_lag_5d",	"perf_stoch14_5d","perf_stoch14_signal_5d",
"perf_stoch14_dif_5d","perf_adx14_neg_5d",	"perf_adx14_pos_5d",	
"perf_williamsr_14_5d","williamsr_14_perf_10d","williamsr_14"	]
    df_temp=df.copy()
    for col in df_temp.columns:
        if col in list_zero:
            df_temp[col]=df_temp[col].apply(lambda x:round(x,digits-2))
        else:
            df_temp[col]=df_temp[col].apply(lambda x:round(x,digits))
    return df_temp


df_x_train, col_y_train = sm.split_df_x_y(
    df_in=df_class, list_features=list_feat, str_label=label, drop_na=True)

df_x_val, col_y_val = sm.split_df_x_y(
    df_in=df_class_val, list_features=list_feat, str_label=label, drop_na=True)

#  for classification datasets
method = RandomUnderSampler(sampling_strategy={0:97300,1:97300,2:97300,3:97300,4:97300}) # 97300 pour lab 20 et 97300 pour lab 50
df_x_train, col_y_train=  method.fit_resample(df_x_train, col_y_train)

method = RandomUnderSampler(sampling_strategy={0:16850,1:16850,2:16850,3:16850,4:16850}) # 16850 pour lab 20 et 14750 pour lab 50
df_x_val, col_y_val=  method.fit_resample(df_x_val, col_y_val)

print(f"{df_x_train.shape=} {df_x_val.shape=} {col_y_train.size=} {col_y_val.size=}")

nb_digits = 3
# round df_x_train and df_x_val
# df_x_train_r = df_x_train.map(lambda x:round_to_significant_digits(x,nb_digits),na_action='ignore')
# df_x_val_r = df_x_val.map(lambda x:round_to_significant_digits(x,nb_digits),na_action='ignore')

df_x_train_r = round_to_significant_digits(df_x_train, nb_digits)
df_x_val_r = round_to_significant_digits(df_x_val, nb_digits)

# save objects in a pickle files
joblib.dump(df_x_train_r, PATH_DATA_DTS+dts_name+"_X_TRAIN_FILT_R"+str(nb_digits)+".pkl")
joblib.dump(col_y_train, PATH_DATA_DTS+dts_name+"_Y_TRAIN_FILT_R"+str(nb_digits)+".pkl")
joblib.dump(df_x_val_r, PATH_DATA_DTS+dts_name+"_X_VAL_FILT_R"+str(nb_digits)+".pkl")
joblib.dump(col_y_val, PATH_DATA_DTS+dts_name+"_Y_VAL_FILT_R"+str(nb_digits)+".pkl")



df_x_train.shape=(486500, 106) df_x_val.shape=(84250, 106) col_y_train.size=486500 col_y_val.size=84250


['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_20D_V4_Y_VAL_FILT_R3.pkl']

In [9]:
dts_name="PARIS_TREND_1D_V5_class_R3_CLIP"
df_class=pd.read_pickle(PATH_DATA_DTS+dts_name+"_TRAIN.pkl")
df_class_val=pd.read_pickle(PATH_DATA_DTS+dts_name+"_VAL.pkl")

print(f"{df_class.shape} {df_class_val.shape}")

(455900, 127) (98665, 127)


In [10]:
# remove useless columns from df_class
list_rem=['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME',
'lab_perf_20d','lab_perf_50d','lab_perf_10d','lab_perf_5d', 'lab_perf_5d','CODE',
'lab_perf_5d_class_3',
'lab_perf_5d_class_5',
'lab_perf_5d_class_10',
'lab_perf_10d_class_3',
'lab_perf_10d_class_5',
'lab_perf_10d_class_10',
'lab_perf_20d_class_3',
'lab_perf_20d_class_5', # the label to predict
# 'lab_perf_20d_class_10',
'lab_perf_50d_class_3',
'lab_perf_50d_class_5',
'lab_perf_50d_class_10']

# remove columns from df_x_train_r don't crash if not in list
df_class.drop(columns=list_rem, inplace=True, errors='ignore')
df_class_val.drop(columns=list_rem, inplace=True, errors='ignore')

print(f"{df_class.shape} {df_class_val.shape}")

(455900, 107) (98665, 107)


In [ ]:
label='lab_perf_20d_class_10'
print(df_class[label].value_counts().sort_index()) # min 43939  >> 43900
print(df_class_val[label].value_counts().sort_index()) # min 8129 >> 8100

lab_perf_20d_class_10
0.0    46448
1.0    45837
2.0    45473
3.0    45017
4.0    43939
5.0    45248
6.0    45673
7.0    46008
8.0    46364
9.0    45893
Name: count, dtype: int64
lab_perf_20d_class_10
0.0    14521
1.0    10314
2.0     9859
3.0     9863
4.0     9697
5.0     9616
6.0     9545
7.0     8830
8.0     8129
9.0     8291
Name: count, dtype: int64


In [12]:
# UNDERSAMPLE THE TRAIN AND VAL AND SAVE DATASET FOR TRAINING
list_feat=df_class.columns.values.tolist()
list_feat.remove(label)

df_x_train, col_y_train = sm.split_df_x_y(
    df_in=df_class, list_features=list_feat, str_label=label, drop_na=True)

df_x_val, col_y_val = sm.split_df_x_y(
    df_in=df_class_val, list_features=list_feat, str_label=label, drop_na=True)

#  for classification datasets
lim_train=43900
lim_val=8100
method = RandomUnderSampler(sampling_strategy={0:lim_train,1:lim_train,2:lim_train,3:lim_train,4:lim_train,
                                               5:lim_train,6:lim_train,7:lim_train,8:lim_train,9:lim_train}) 
df_x_train, col_y_train=  method.fit_resample(df_x_train, col_y_train)

method = RandomUnderSampler(sampling_strategy={0:lim_val,1:lim_val,2:lim_val,3:lim_val,4:lim_val,
                                               5:lim_val,6:lim_val,7:lim_val,8:lim_val,9:lim_val}) 
df_x_val, col_y_val=  method.fit_resample(df_x_val, col_y_val)

joblib.dump(df_x_train, PATH_DATA_DTS+dts_name+"_"+label+"_X_TRAIN.pkl")
joblib.dump(col_y_train, PATH_DATA_DTS+dts_name+"_"+label+"_Y_TRAIN.pkl")
joblib.dump(df_x_val, PATH_DATA_DTS+dts_name+"_"+label+"_X_VAL.pkl")
joblib.dump(col_y_val, PATH_DATA_DTS+dts_name+"_"+label+"_Y_VAL.pkl")


['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_V5_class_R3_CLIP_lab_perf_20d_class_10_Y_VAL.pkl']

In [ ]:
# #ONLY FIRST TIME 
# list_rem=['perf_pos_ema20_10d','perf_pos_ema20_5d','perf_stoch14_5d','perf_stoch14_dif_5d'
# ,'perf_williamsr_14_5d','pos_ema10','pos_ema5','pos_sma10','ret_1d','stoch14_dif','perf_pos_ema200_10d'
# ,'pos_avg_vol14','aroon14_down','ret_10d','perf_sma10_5d','perf_adx14_neg_5d','ret_5d'
# ,'perf_sma5_5d','perf_pos_ema200_5d','perf_pos_ema10_5d','perf_sma_10_5d','perf_pos_sma10_10d'
# ,'stdev20_1d','perf_adx14_pos_5d','perf_pos_ema10_10d','perf_stoch14_signal_5d','pos_sma5_10'
# ,'tr_atr14','pos_ema50_perf_5d','pos_ema50_perf_10d','pos_rsi14_sma5','williamsr_14_perf_10d'
# ,'rsi14','perf_pos_sma200_10d','perf_pos_sma50_10d','pos_sma5','aroon14_up_lag_5d'
# ,'pos_rsi14_sma20','pos_sma10_20','perf_rsi14_10d','perf_ema5_5d','aroon14_dif_lag_5d'
# ,'perf_adx14_dif_5d','pos_bb20_hi_lag_5d','perf_pos_sma20_10d','aroon14_dif'
# ,'pos_sma5_50','pos_rsi14_sma5_20','perf_rsi14_5d','sma5_rsi14','pos_sma20'
# ,'stoch14','perf_sma10_10d','pos_sma5_20','pos_bb20_hi','pos_sma10_50','pos_stdev20_sma5'
# ,'perf_sma20_5d','perf_sma50_5d','pos_sma50','aroon14_down_lag_5d','pos_bb20_lo_lag_5d'
# ,'pos_bot20','pos_top20','pos_top20_lag_5d','perf_sma_20_5d','pos_bot20_lag_5d','perf_sma_200_5d','pos_sma50_200'
# ]

# # remove columns from df_x_train_r don't crash if not in list
# df_x_train_r.drop(columns=list_rem, inplace=True, errors='ignore')
# df_x_val_r.drop(columns=list_rem, inplace=True, errors='ignore')

# print(df_x_train_r.shape)
# print(df_x_val_r.shape)

# # save the 2 df in pkl format
# joblib.dump(df_x_train_r, PATH_DATA_DTS+dts_name+"_X_TRAIN_FILT_R"+nb_digits+".pkl")
# joblib.dump(df_x_val_r, PATH_DATA_DTS+dts_name+"_X_VAL_FILT_R"+nb_digits+".pkl")


(486879, 39)
(102506, 39)


['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_20D_REG_V4_X_VAL_FILT_R3.pkl']

In [4]:
# drop from df_x_train_r and df_x_val_r the columns pos_sma50_200 and pos_sma20_200
# save list of columns of df_x_train_r in an info file   
nb_digits="3"
with open(PATH_DATA_DTS+dts_name+"_X_TRAIN_FILT_R"+nb_digits+"_INFO.txt", "w") as f:
    for s in df_x_train_r.columns:
        f.write(str(s) +"\n")

In [3]:
# Load datasets before training
dts_name="PARIS_TREND_1D_V5_class_R3_CLIP"
label="lab_perf_20d_class_10"

df_x_train_r = joblib.load(PATH_DATA_DTS+dts_name+"_"+label+"_X_TRAIN.pkl")
col_y_train = joblib.load(PATH_DATA_DTS+dts_name+"_"+label+"_Y_TRAIN.pkl")
df_x_val_r = joblib.load(PATH_DATA_DTS+dts_name+"_"+label+"_X_VAL.pkl")
col_y_val = joblib.load(PATH_DATA_DTS+dts_name+"_"+label+"_Y_VAL.pkl")

print(f"{df_x_train_r.shape=} {df_x_val_r.shape=} {col_y_train.size=} {col_y_val.size=}")

df_x_train_r.shape=(439000, 106) df_x_val_r.shape=(81000, 106) col_y_train.size=439000 col_y_val.size=81000


In [4]:
list_keep=['pos_bot_200','pos_ema50_200','adx14','macd_dif','pos_sma50_200','pos_top_200','adx14_neg'
,'pos_bot50','adx14_pos','perf_sma200_10d','macd_fast','pos_stdev20_sma20','pos_bot50_lag_10d','perf_sma50_10d'
,'sma20_rsi14','pos_sma20_200','perf_adx14_5d','macd_slow','perf_atr14_5d','pos_sma200','pos_sma20_50','adx14_dif'
,'pos_bot20_lag_20d','perf_sma20_10d','pos_top50','pos_avg_vol50','stoch14_signal','sma5_rsi14','pos_rsi14_sma5_20'
,'pos_sma10_200','pos_top50_lag_10d','pos_bb20_hi_lag_5d','pos_top20_lag_20d','pos_sma5_200','pos_bot20'
,'stoch14','pos_bb20_lo_lag_5d','pos_stdev20_sma5','trix12','aroon14_dif','pos_bb20_hi','rsi14','perf_adx14_neg_5d'
,'tr_atr14','williamsr_14','pos_bb20_lo','pos_sma50','perf_adx14_pos_5d','perf_stoch14_dif_5d','ret_20d'
,'pos_bot20_lag_5d','pos_sma10_50','pos_sma5_50','stdev20_1d','aroon14_dif_lag_5d','pos_top20','perf_sma200_5d'
,'perf_sma_200_5d','perf_sma50_5d']

# remove columns from df_x_train_r idf not in list_keep, don't crash if not in list
df_x_train_r.drop(columns=[col for col in df_x_train_r.columns if col not in list_keep], inplace=True, errors='ignore')
df_x_val_r.drop(columns=[col for col in df_x_val_r.columns if col not in list_keep], inplace=True, errors='ignore')

print(f"{df_x_train_r.shape=} {df_x_val_r.shape=} {col_y_train.size=} {col_y_val.size=}")

df_x_train_r.shape=(439000, 59) df_x_val_r.shape=(81000, 59) col_y_train.size=439000 col_y_val.size=81000


In [ ]:
# print df_x_train_r describe() but with all the columns
pd.set_option('display.max_columns', None)
print(df_x_train_r.describe())

LIGHTGBM

In [ ]:
# TODO : put this in a library
# lgbm_mngr.py 

import lightgbm as lgb
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

def custom_penalty_score(y_true, y_pred):
    # print(f"{y_true.shape=} {y_pred.shape=}")
    if isinstance(y_true, lgb.Dataset):
        y_true = y_true.get_label()
    # Convert predicted probabilities to class labels
    if y_pred.ndim == 1:
        # raise ValueError(f"Expected 2D array for y_pred, got 1D array with shape {y_pred.shape}")
        y_pred_labels = y_pred
    else :
        y_pred_labels = np.argmax(y_pred, axis=1)  # Convert predicted probabilities to class labels
    if y_true.shape[0] != y_pred_labels.shape[0]:
        raise ValueError(f"Shape mismatch: y_true has shape {y_true.shape} but y_pred_labels has shape {y_pred_labels.shape}")
  
    error = (y_true - y_pred_labels) ** 2
    return 'custom_penalty', error.mean(), False  # Return the mean squared error (lower is better)

# Create a scorer object
custom_scorer = metrics.make_scorer(custom_penalty_score, is_higher_better=False)

def print_eval_metric(env):
    # env is a callback environment that contains information about the training process
    result = env.evaluation_result_list
    for item in result:
        metric_name, dataset_name, metric_value, is_higher_better = item
        print(f"Iteration {env.iteration}: {dataset_name} - {metric_name} = {metric_value}")

def calculate_tree_depth(tree):
    if 'leaf_index' in tree:
        return 0
    else:
        left_depth = calculate_tree_depth(tree['left_child'])
        right_depth = calculate_tree_depth(tree['right_child'])
        return 1 + max(left_depth, right_depth)


In [ ]:
# for classification datasets !!!!!!!!!

device_type='gpu'
results_df = pd.DataFrame(columns=['params','accuracy_train', 'accuracy_val','penalty_score',
                                   'precision','recall','best_iter','seed','max_max_depth','avg_max_depth'])

params = {
    'learning_rate': [0.0001],  # 0.0005 ?
    'max_depth': [20,25],  # np.linspace(11,15,3, endpoint=True,dtype=int),
    'num_leaves': [47,63,79],  # 1023,1536],#np.linspace(15, 60,10, endpoint=True,dtype=int),
    'min_data_in_leaf': [50],
    'feature_fraction': [0.25],
    'lambda_l1': [0.1,0.2],#[0,0.1,1,5] np.exp(np.random.default_rng().uniform(np.log(1e-4), np.log(10), 20)),
    'lambda_l2': [0.5,1],#[0,0.1,1,5]  np.exp(np.random.default_rng().uniform(np.log(1e-4), np.log(10), 20)),
}

nb_class=10
nb_seeds=3
n_splits=3
method='strat'

if method=='time':
    scv = TimeSeriesSplit(n_splits=1)
elif method=='strat':
    scv = StratifiedKFold(n_splits=n_splits, shuffle=False)
else:
    raise ValueError(f"Unknown method: {method}")

# Generate all combinations of parameters
keys, values = zip(*params.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

# Print the list of parameter combinations
print(f"{combinations.__len__()=}")
for combo in combinations[:2]:  # Print only the first 3 combinations for brevity
    print(combo)

# params=[
#     {'num_leaves': 511, 'min_data_in_leaf': 100, 'max_depth': 11, 'learning_rate': 0.01, 'lambda_l2': 0.025, 'lambda_l1': 0.01, 'feature_fraction': 0.4},
    # {'num_leaves': 1023, 'min_data_in_leaf': 100, 'max_depth': 15, 'learning_rate': 0.01, 'lambda_l2': 0.5, 'lambda_l1': 0.25, 'feature_fraction': 0.6}
#        ]

seeds=np.random.default_rng().integers(0,1000,nb_seeds)

print(f"{seeds=}")
# loop over the parameters
for param in combinations:
    try:
        for seed in seeds:
            print(f"{dt.now().strftime('%Y-%m-%d %H:%M:%S')} Try : {param}")
            clf=lgb.LGBMClassifier(objective='multiclass',metric='custom',device_type=device_type,verbosity=-1,
                                seed=seed, num_class=nb_class,n_estimators=1000,#np.random.default_rng().integers(0,10000),
                                **param)
            
            for train_index, val_index in scv.split(df_x_train_r, col_y_train):
                X_train, X_val = df_x_train_r.iloc[train_index], df_x_train_r.iloc[val_index]
                y_train, y_val = col_y_train.iloc[train_index], col_y_train.iloc[val_index]

                # train the model with the best parameters and eval set without log
                clf.fit(X_train, y_train,eval_set=[(X_val, y_val)],eval_metric=custom_penalty_score,
                        callbacks=[lgb.early_stopping(stopping_rounds=20)])#print_eval_metric

                print(f"{dt.now().strftime('%Y-%m-%d %H:%M:%S')} Best iteration: {clf.best_iteration_}")

            y_pred = clf.predict(df_x_train_r)
            accuracy_train = accuracy_score(col_y_train, y_pred)
            penalty_score=custom_penalty_score(col_y_train, y_pred)
            # print(f'Accuracy Train: {accuracy_train} Penalty Train: {penalty_score}')

            y_pred = clf.predict(df_x_val_r)
            accuracy_val = accuracy_score(col_y_val, y_pred)
            penalty_score=custom_penalty_score(col_y_val, y_pred)
            # print(f'Accuracy Val: {accuracy_val} Penalty Val: {penalty_score}')
            
            # Extract precision values
            precision_vals = precision_score(col_y_val, y_pred, average=None)
            precision_str ='precision:'+'-'.join([f'{i}:{precision_vals[i]:.2f}' for i in range(len(precision_vals))])
            
            # Extract recall values
            recall_vals = recall_score(col_y_val, y_pred, average=None)
            recall_str = 'recall:'+'-'.join([f'{i}:{recall_vals[i]:.2f}' for i in range(len(recall_vals))])

            # print(classification_report(col_y_val, y_pred))
            # print the confusion matrix
            print(metrics.confusion_matrix(col_y_val, y_pred))

            # Calculate the depth of each tree
            booster = clf.booster_
            tree_info = booster.dump_model()["tree_info"]
            # print(tree_info)
            max_depths = [calculate_tree_depth(tree["tree_structure"]) for tree in tree_info]
            avg_max_depth = sum(max_depths) / len(max_depths)
            max_max_depth = max(max_depths)
            # print(f"Max depth of trees: {avg_max_depth=} {max_max_depth=}")


            # save the values of param and accuracy_val in a dataframe
            results_df = pd.concat([results_df, 
                                    pd.DataFrame([{'params': param,'accuracy_train':round(accuracy_train,3),
                                                    'accuracy_val': round(accuracy_val,3),'penalty_score':round(penalty_score[1],3),
                                                    'precision':precision_str,'recall':recall_str,'best_iter':clf.best_iteration_,
                                                    'seed':seed,'max_max_depth':max_max_depth,'avg_max_depth':avg_max_depth}])],
                                                    ignore_index=True)
    except Exception as e:
        print(f"Error with parameters: {param}")
        print(f"Error message: {e}")   

    

# save tje results in a file
results_df.to_csv(PATH_DATA_DTS+dts_name+"_LGBM_RESULTS_CLASS_10_V5_c.csv", sep=",")
# save the model

# joblib.dump(clf, PATH_DATA_DTS+dts_name+"_LGBM_MODEL_R2.pkl")


In [7]:
results_df.to_csv(PATH_DATA_DTS+dts_name+"_LGBM_RESULTS_CLASS_10_V5_a.csv", sep=",")

In [8]:
#  TEST 1 SET OF PARAMETERS for classification datasets !!!!!!!!!
device_type='gpu'
param={'learning_rate': 0.0001, 'max_depth': 20, 'num_leaves': 47, 'min_data_in_leaf': 50, 'feature_fraction': 0.25, 'lambda_l1': 0.2, 'lambda_l2': 1}
seed=464
nb_class=10
n_splits=3
method='strat'

if method=='time':
    scv = TimeSeriesSplit(n_splits=1)
elif method=='strat':
    scv = StratifiedKFold(n_splits=n_splits, shuffle=False)
else:
    raise ValueError(f"Unknown method: {method}")

clf=lgb.LGBMClassifier(objective='multiclass',metric='custom',device_type=device_type,verbosity=-1,
                                seed=seed, num_class=nb_class,n_estimators=1000,#np.random.default_rng().integers(0,10000),
                                **param)

for train_index, val_index in scv.split(df_x_train_r, col_y_train):
                X_train, X_val = df_x_train_r.iloc[train_index], df_x_train_r.iloc[val_index]
                y_train, y_val = col_y_train.iloc[train_index], col_y_train.iloc[val_index]

                # train the model with the best parameters and eval set without log
                clf.fit(X_train, y_train,eval_set=[(X_val, y_val)],eval_metric=custom_penalty_score,
                        callbacks=[lgb.early_stopping(stopping_rounds=20)])#print_eval_metric
# clf.fit(df_x_train_r, col_y_train, eval_set=[(df_x_val_r, col_y_val)],eval_metric=custom_penalty_score,
#         callbacks=[lgb.early_stopping(stopping_rounds=20)])#print_eval_metric

y_pred = clf.predict(df_x_train_r)
accuracy_train = accuracy_score(col_y_train, y_pred)
penalty_score=custom_penalty_score(col_y_train, y_pred)
# print(f'Accuracy Train: {accuracy_train} Penalty Train: {penalty_score}')

y_pred = clf.predict(df_x_val_r)
accuracy_val = accuracy_score(col_y_val, y_pred)
penalty_score=custom_penalty_score(col_y_val, y_pred)
print(f'Accuracy Val: {accuracy_val} Penalty Val: {penalty_score}')

# Extract precision values
precision_vals = precision_score(col_y_val, y_pred, average=None)
precision_str ='precision:'+'-'.join([f'{i}:{precision_vals[i]:.2f}' for i in range(len(precision_vals))])
print(precision_str)

# Extract recall values
recall_vals = recall_score(col_y_val, y_pred, average=None)
recall_str = 'recall:'+'-'.join([f'{i}:{recall_vals[i]:.2f}' for i in range(len(recall_vals))])
print(recall_str)

print(metrics.confusion_matrix(col_y_val, y_pred))

# Calculate the depth of each tree
booster = clf.booster_
tree_info = booster.dump_model()["tree_info"]
# print(tree_info)
max_depths = [calculate_tree_depth(tree["tree_structure"]) for tree in tree_info]
avg_max_depth = sum(max_depths) / len(max_depths)
max_max_depth = max(max_depths)
print(f"Max depth of trees: {avg_max_depth=} {max_max_depth=}")



Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's custom_penalty: 15.829
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[13]	valid_0's custom_penalty: 16.6834
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[95]	valid_0's custom_penalty: 16.6917
Accuracy Val: 0.14679012345679013 Penalty Val: ('custom_penalty', 17.555234567901234, False)
precision:0:0.16-1:0.12-2:0.11-3:0.13-4:0.17-5:0.13-6:0.12-7:0.11-8:0.13-9:0.21
recall:0:0.35-1:0.06-2:0.05-3:0.08-4:0.17-5:0.17-6:0.12-7:0.09-8:0.11-9:0.26
[[2808  447  297  320  362  636  627  621  623 1359]
 [1806  476  453  473  710  964  818  760  622 1018]
 [1407  433  440  619  972 1159  903  707  618  842]
 [1223  375  397  663 1085 1287  953  722  596  799]
 [1131  335  399  590 1379 1365  874  716  608  703]
 [1117  315  431  658 1104 1401  957  787  659  671]
 [1332  387  416  585  915 13

In [7]:
# calculate the mean and std of col_y_train and col_y_val
print(f"{col_y_train.mean()} {col_y_train.std()}")
print(f"{col_y_val.mean()} {col_y_val.std()}")

4.5 2.8722845946656324
4.5 2.872299053564809


In [9]:
#  SAVE THE MODEL

model_name=dts_name+"_LGBM_MODEL_CLAS10_R3_2503_V1"
joblib.dump(clf, PATH_DATA_DTS+model_name+".pkl")

#  Save the metadata of the model in a file : all the params and the results
metadata = {'params': param,'accuracy_train':round(accuracy_train,3),
            'accuracy_val': round(accuracy_val,3),'penalty_score':round(penalty_score[1],3),
            'precision':precision_str,'recall':recall_str,'best_iter':clf.best_iteration_,
            'seed':seed,'max_max_depth':max_max_depth,'avg_max_depth':avg_max_depth}
#  in metadata, add the list of features and label
metadata['features'] = df_x_train_r.columns
metadata['label'] = label

# save the metadata in a file
with open(PATH_DATA_DTS+model_name+".txt", "w") as f:
    for key in metadata:
        f.write(f"{key} : {metadata[key]}\n")


In [ ]:
#  print the feature importance
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,df_x_train_r.columns)), columns=['Value','Feature'])
plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

# save the feature importance in a file
feature_imp.to_csv(PATH_DATA_DTS+dts_name+"_LGBM_FEATURES_D.csv", sep=",")


In [ ]:
# print la map of the model
lgb.plot_importance(clf,importance_type='split',max_num_features=10,figsize=(20,10),title='Feature importance(split)')
lgb.plot_importance(clf,importance_type='gain',max_num_features=10,figsize=(20,10),title='Feature importance(gain)')
plt.show()


In [ ]:
y_pred = clf.predict(df_x_train_r)
accuracy = accuracy_score(col_y_train, y_pred)
print(f'Accuracy Train: {accuracy}')


y_pred = clf.predict(df_x_val_r)
accuracy = accuracy_score(col_y_val, y_pred)
print(f'Accuracy Val: {accuracy}')
print(classification_report(col_y_val, y_pred))
# print the confusion matrix
print(metrics.confusion_matrix(col_y_val, y_pred))

In [ ]:

lgb.plot_importance(clf,max_num_features=40, figsize=(7,6), title="LightGBM Feature Importance")
plt.show()
# save features importance in a file
feature_imp=pd.DataFrame()
feature_imp['features']=df_x_train_r.columns
feature_imp['importance']=clf.feature_importances_
feature_imp.to_csv(PATH_DATA_DTS+dts_name+"_LGBM_FEATURE_IMPORTANCE.csv", sep=",")


In [11]:
# load the model PARIS_TREND_1D_20D_V3_LGBM_MODEL_R2.pkl and print the characteristics
clf = joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5_class_R3_CLIP_LGBM_MODEL_CLAS5_R3_2503_V1.pkl")
print(clf)


LGBMClassifier(device_type='gpu', feature_fraction=0.25, lambda_l1=0,
               lambda_l2=0.1, learning_rate=0.0005, max_depth=25,
               metric='custom', min_data_in_leaf=100, n_estimators=1000,
               num_class=5, num_leaves=63, objective='multiclass', seed=323,
               verbosity=-1)


PREPARATION FOR BACKTEST

In [12]:
# Load datasets before training
dts_name="PARIS_TREND_1D_V5_class_R3_CLIP"

df_train = joblib.load(PATH_DATA_DTS+dts_name+"_TRAIN.pkl")
df_val = joblib.load(PATH_DATA_DTS+dts_name+"_VAL.pkl")
df_conf = joblib.load(PATH_DATA_DTS+dts_name+"_CONF.pkl")

print(f"{df_train.shape=} {df_val.shape=} {df_conf.shape=}")

df_train.shape=(455900, 127) df_val.shape=(98665, 127) df_conf.shape=(117005, 127)


In [13]:
label="lab_perf_20d_class_10"

df_train_w=df_train.copy()
df_val_w=df_val.copy()
df_conf_w=df_conf.copy()

list_rem=['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME',
'lab_perf_20d','lab_perf_50d','lab_perf_10d','lab_perf_5d', 'lab_perf_5d','CODE',
'lab_perf_5d_class_3',
'lab_perf_5d_class_5',
'lab_perf_5d_class_10',
'lab_perf_10d_class_3',
'lab_perf_10d_class_5',
'lab_perf_10d_class_10',
'lab_perf_20d_class_3',
'lab_perf_20d_class_5', # the label to predict
# 'lab_perf_20d_class_10',
'lab_perf_50d_class_3',
'lab_perf_50d_class_5',
'lab_perf_50d_class_10']

# remove columns from df_x_train_r don't crash if not in list
df_train_w.drop(columns=list_rem, inplace=True, errors='ignore')
df_val_w.drop(columns=list_rem, inplace=True, errors='ignore')
df_conf_w.drop(columns=list_rem, inplace=True, errors='ignore')

print(f"{df_train_w.shape} {df_val_w.shape} {df_conf_w.shape}")

list_feat=df_train_w.columns.values.tolist()
list_feat.remove(label)

df_x_train_10, col_y_train_10 = sm.split_df_x_y(
    df_in=df_train_w, list_features=list_feat, str_label=label, drop_na=False)

df_x_val_10, col_y_val_10 = sm.split_df_x_y(
    df_in=df_val_w, list_features=list_feat, str_label=label, drop_na=False)

df_x_conf_10, col_y_conf_10 = sm.split_df_x_y(
    df_in=df_conf_w, list_features=list_feat, str_label=label, drop_na=False)


list_keep=['pos_bot_200','pos_ema50_200','adx14','macd_dif','pos_sma50_200','pos_top_200','adx14_neg'
,'pos_bot50','adx14_pos','perf_sma200_10d','macd_fast','pos_stdev20_sma20','pos_bot50_lag_10d','perf_sma50_10d'
,'sma20_rsi14','pos_sma20_200','perf_adx14_5d','macd_slow','perf_atr14_5d','pos_sma200','pos_sma20_50','adx14_dif'
,'pos_bot20_lag_20d','perf_sma20_10d','pos_top50','pos_avg_vol50','stoch14_signal','sma5_rsi14','pos_rsi14_sma5_20'
,'pos_sma10_200','pos_top50_lag_10d','pos_bb20_hi_lag_5d','pos_top20_lag_20d','pos_sma5_200','pos_bot20'
,'stoch14','pos_bb20_lo_lag_5d','pos_stdev20_sma5','trix12','aroon14_dif','pos_bb20_hi','rsi14','perf_adx14_neg_5d'
,'tr_atr14','williamsr_14','pos_bb20_lo','pos_sma50','perf_adx14_pos_5d','perf_stoch14_dif_5d','ret_20d'
,'pos_bot20_lag_5d','pos_sma10_50','pos_sma5_50','stdev20_1d','aroon14_dif_lag_5d','pos_top20','perf_sma200_5d'
,'perf_sma_200_5d','perf_sma50_5d']

# remove columns from df_x_train_r idf not in list_keep, don't crash if not in list
df_x_train_10.drop(columns=[col for col in df_x_train_10.columns if col not in list_keep], inplace=True, errors='ignore')
df_x_val_10.drop(columns=[col for col in df_x_val_10.columns if col not in list_keep], inplace=True, errors='ignore')
df_x_conf_10.drop(columns=[col for col in df_x_conf_10.columns if col not in list_keep], inplace=True, errors='ignore')

print(f"{df_x_train_10.shape=} {df_x_val_10.shape=} {df_x_conf_10.shape=} {col_y_train_10.size=} {col_y_val_10.size=} {col_y_conf_10.size=}")

(455900, 107) (98665, 107) (117005, 107)
df_x_train_10.shape=(455900, 59) df_x_val_10.shape=(98665, 59) df_x_conf_10.shape=(117005, 59) col_y_train_10.size=455900 col_y_val_10.size=98665 col_y_conf_10.size=117005


In [15]:
label="lab_perf_20d_class_5"

df_train_w=df_train.copy()
df_val_w=df_val.copy()
df_conf_w=df_conf.copy()

list_rem=['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME',
'lab_perf_20d','lab_perf_50d','lab_perf_10d','lab_perf_5d', 'lab_perf_5d','CODE',
'lab_perf_5d_class_3',
'lab_perf_5d_class_5',
'lab_perf_5d_class_10',
'lab_perf_10d_class_3',
'lab_perf_10d_class_5',
'lab_perf_10d_class_10',
'lab_perf_20d_class_3',
# 'lab_perf_20d_class_5', # the label to predict
'lab_perf_20d_class_10',
'lab_perf_50d_class_3',
'lab_perf_50d_class_5',
'lab_perf_50d_class_10']

# remove columns from df_x_train_r don't crash if not in list
df_train_w.drop(columns=list_rem, inplace=True, errors='ignore')
df_val_w.drop(columns=list_rem, inplace=True, errors='ignore')
df_conf_w.drop(columns=list_rem, inplace=True, errors='ignore')

print(f"{df_train_w.shape} {df_val_w.shape} {df_conf_w.shape}")

list_feat=df_train_w.columns.values.tolist()
list_feat.remove(label)

df_x_train_5, col_y_train_5 = sm.split_df_x_y(
    df_in=df_train_w, list_features=list_feat, str_label=label, drop_na=False)

df_x_val_5, col_y_val_5 = sm.split_df_x_y(
    df_in=df_val_w, list_features=list_feat, str_label=label, drop_na=False)

df_x_conf_5, col_y_conf_5 = sm.split_df_x_y(
    df_in=df_conf_w, list_features=list_feat, str_label=label, drop_na=False)


list_keep=['pos_bot_200','pos_ema50_200','adx14','macd_dif','pos_sma50_200','pos_top_200','adx14_neg'
,'pos_bot50','adx14_pos','perf_sma200_10d','macd_fast','pos_stdev20_sma20','pos_bot50_lag_10d','perf_sma50_10d'
,'sma20_rsi14','pos_sma20_200','perf_adx14_5d','macd_slow','perf_atr14_5d','pos_sma200','pos_sma20_50','adx14_dif'
,'pos_bot20_lag_20d','perf_sma20_10d','pos_top50','pos_avg_vol50','stoch14_signal','sma5_rsi14','pos_rsi14_sma5_20'
,'pos_sma10_200','pos_top50_lag_10d','pos_bb20_hi_lag_5d','pos_top20_lag_20d','pos_sma5_200','pos_bot20'
,'stoch14','pos_bb20_lo_lag_5d','pos_stdev20_sma5','trix12','aroon14_dif','pos_bb20_hi','rsi14','perf_adx14_neg_5d'
,'tr_atr14','williamsr_14','pos_bb20_lo','pos_sma50','perf_adx14_pos_5d','perf_stoch14_dif_5d','ret_20d'
,'pos_bot20_lag_5d','pos_sma10_50','pos_sma5_50','stdev20_1d','aroon14_dif_lag_5d','pos_top20','perf_sma200_5d'
,'perf_sma_200_5d','perf_sma50_5d']

# remove columns from df_x_train_r idf not in list_keep, don't crash if not in list
df_x_train_5.drop(columns=[col for col in df_x_train_5.columns if col not in list_keep], inplace=True, errors='ignore')
df_x_val_5.drop(columns=[col for col in df_x_val_5.columns if col not in list_keep], inplace=True, errors='ignore')
df_x_conf_5.drop(columns=[col for col in df_x_conf_5.columns if col not in list_keep], inplace=True, errors='ignore')

print(f"{df_x_train_5.shape=} {df_x_val_5.shape=} {df_x_conf_5.shape=} {col_y_train_5.size=} {col_y_val_5.size=} {col_y_conf_5.size=}")

(455900, 107) (98665, 107) (117005, 107)
df_x_train_5.shape=(455900, 59) df_x_val_5.shape=(98665, 59) df_x_conf_5.shape=(117005, 59) col_y_train_5.size=455900 col_y_val_5.size=98665 col_y_conf_5.size=117005


In [17]:
# Check if the indices are aligned
assert df_train.index.equals(df_x_train_10.index), "Row order is not consistent!"
assert df_train.index.equals(df_x_train_5.index), "Row order is not consistent!"
assert df_val.index.equals(df_x_val_10.index), "Row order is not consistent!"
assert df_val.index.equals(df_x_val_5.index), "Row order is not consistent!"
assert df_conf.index.equals(df_x_conf_10.index), "Row order is not consistent!"
assert df_conf.index.equals(df_x_conf_5.index), "Row order is not consistent!"

In [8]:
clf_10 = joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5_class_R3_CLIP_LGBM_MODEL_CLAS10_R3_2503_V1.pkl")
print(clf_10)

clf_5 = joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5_class_R3_CLIP_LGBM_MODEL_CLAS5_R3_2503_V1.pkl")
print(clf_5)



LGBMClassifier(device_type='gpu', feature_fraction=0.25, lambda_l1=0.2,
               lambda_l2=1, learning_rate=0.0001, max_depth=20, metric='custom',
               min_data_in_leaf=50, n_estimators=1000, num_class=10,
               num_leaves=47, objective='multiclass', seed=464, verbosity=-1)
LGBMClassifier(device_type='gpu', feature_fraction=0.25, lambda_l1=0,
               lambda_l2=0.1, learning_rate=0.0005, max_depth=25,
               metric='custom', min_data_in_leaf=100, n_estimators=1000,
               num_class=5, num_leaves=63, objective='multiclass', seed=323,
               verbosity=-1)


In [19]:
# predict the values of the 2 models
df_train['predict_10']=clf_10.predict(df_x_train_10)
df_train['predict_5']=clf_5.predict(df_x_train_5)
df_val['predict_10']=clf_10.predict(df_x_val_10)
df_val['predict_5']=clf_5.predict(df_x_val_5)
df_conf['predict_10']=clf_10.predict(df_x_conf_10)
df_conf['predict_5']=clf_5.predict(df_x_conf_5)

df_train['predict_10_proba'] = clf_10.predict_proba(df_x_train_10).tolist()
df_train['predict_5_proba'] = clf_5.predict_proba(df_x_train_5).tolist()
df_val['predict_10_proba'] = clf_10.predict_proba(df_x_val_10).tolist()
df_val['predict_5_proba'] = clf_5.predict_proba(df_x_val_5).tolist()
df_conf['predict_10_proba'] = clf_10.predict_proba(df_x_conf_10).tolist()
df_conf['predict_5_proba'] = clf_5.predict_proba(df_x_conf_5).tolist()

print(f"{df_train.shape=} {df_val.shape=} {df_conf.shape=}")

df_train.shape=(455900, 131) df_val.shape=(98665, 131) df_conf.shape=(117005, 131)


In [20]:
df_train['PART']='TRAIN'
df_val['PART']='VAL'
df_conf['PART']='CONF'

df_all=pd.concat([df_train,df_val,df_conf],axis=0)
print(f"{df_all.shape=}")
# save the df_all in a file
joblib.dump(df_all, PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL.pkl")

df_all.shape=(671570, 132)


['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL.pkl']

In [22]:
# print list of columns and index of df_all in a file
with open(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL_INFO.txt", "w") as f:
    f.write(f"Columns:\n{list(df_all.columns)}\n")
    f.write(f"Index:\n{list(df_all.index)}\n")

In [4]:
df_all=joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL.pkl")
print(f"{df_all.shape=}")

df_all.shape=(671570, 132)


In [5]:
df_all_ana=df_all.copy()
col_keep=['pos_sma20_200','predict_10','predict_5','predict_10_proba','predict_5_proba','PART',
          'lab_perf_20d','lab_perf_20d_class_5', 'lab_perf_20d_class_10']
df_all_ana.drop(columns=[col for col in df_all_ana.columns if col not in col_keep], inplace=True, errors='ignore')
print(f"{df_all_ana.shape=}")

df_all_ana.shape=(671570, 9)


In [ ]:
# TODO replace by lgbm_mngr.py
def expand_probabilities(df, column_prefix, proba_column, num_classes):
    """
    Expands a column of probabilities into separate columns for each class.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the probabilities column.
    column_prefix (str): The prefix for the new columns (e.g., 'predict_10_proba').
    proba_column (str): The name of the column containing the probabilities.
    num_classes (int): The number of classes (length of the probability array).

    Returns:
    pd.DataFrame: The updated DataFrame with expanded probability columns.
    """
    for i in range(num_classes):
        df[f'{column_prefix}_{i}'] = df[proba_column].apply(lambda x: x[i]).round(5)
    return df

# Expand probabilities for predict_10_proba
df_all_ana = expand_probabilities(df_all_ana, 'predict_10_proba', 'predict_10_proba', 10)

# Expand probabilities for predict_5_proba
df_all_ana = expand_probabilities(df_all_ana, 'predict_5_proba', 'predict_5_proba', 5)

# Print the first few rows to verify
print(df_all_ana.head())

In [14]:
# if predict_10=9 what is the mean of each predict_10_proba_i
# print(df_all_ana[df_all_ana['predict_10']==9].loc[:, 'predict_10_proba_0':'predict_10_proba_9'].mean())

print(df_all_ana[df_all_ana['predict_10']==9].loc[:, 'predict_10_proba_0':'predict_10_proba_9'].mean())

predict_10_proba_0    0.100285
predict_10_proba_1    0.100040
predict_10_proba_2    0.099894
predict_10_proba_3    0.099835
predict_10_proba_4    0.099813
predict_10_proba_5    0.099797
predict_10_proba_6    0.099845
predict_10_proba_7    0.099877
predict_10_proba_8    0.100016
predict_10_proba_9    0.100598
dtype: float64


In [ ]:
# for class 5 and predict_5 in (3,4), create a new df aggregated by PART, predict_5, lab_perf_20d_class_5
# and calculate the count of line, mean of each predict_5_proba_i
limit =0.202 # keep 0.202 instead of 0.2015
df_w=df_all_ana[df_all_ana['predict_5_proba_4']>limit]
df_all_ana_5=df_w[(df_w['predict_5'].isin([3,4])) ].groupby(['PART','predict_5','lab_perf_20d_class_5']).agg(
    count=('predict_5','count'),
    mean_0=('predict_5_proba_0','mean'),
    mean_1=('predict_5_proba_1','mean'),
    mean_2=('predict_5_proba_2','mean'),
    mean_3=('predict_5_proba_3','mean'),
    mean_4=('predict_5_proba_4','mean')
).reset_index()
df_all_ana_5.to_csv(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_5_3.csv", sep=",")

In [ ]:
# for class 10 and predict_10 in (8,9), create a new df aggregated by PART, predict_10, lab_perf_20d_class_10
# and calculate the count of line, mean of each predict_10_proba_i
limit =0.101 # keep 
df_w = df_all_ana[(df_all_ana['predict_10_proba_9'] > limit) | (df_all_ana['predict_10_proba_8'] > limit)]
df_all_ana_10=df_w[(df_w['predict_10'].isin([8,9])) ].groupby(['PART','predict_10','lab_perf_20d_class_10']).agg(
    count=('predict_10','count'),
    mean_0=('predict_10_proba_0','mean'),
    mean_1=('predict_10_proba_1','mean'),
    mean_2=('predict_10_proba_2','mean'),
    mean_3=('predict_10_proba_3','mean'),
    mean_4=('predict_10_proba_4','mean'),
    mean_5=('predict_10_proba_5','mean'),
    mean_6=('predict_10_proba_6','mean'),
    mean_7=('predict_10_proba_7','mean'),
    mean_8=('predict_10_proba_8','mean'),
    mean_9=('predict_10_proba_9','mean')
).reset_index()

df_all_ana_10.to_csv(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_10_2.csv", sep=",")

In [15]:
df_conf=df_all_ana[df_all_ana['PART']=='CONF']
# df_conf_c5_4=df_conf[df_conf['predict_5']==4]
df_conf_c10_9=df_conf[df_conf['predict_10']==9]


In [ ]:
# print a bar chart from df_conf_c5_4 with a serie by lab_perf_20d_class_5 [0-4] 
# X will the proba in 10 groupsand Y the count of line
# bins = np.linspace(0.199, 0.21, 11)
# df_conf_c5_4['predict_5_proba_4_bins'] = pd.cut(df_conf_c5_4['predict_5_proba_4'], bins=bins, include_lowest=True)
# bar_data = df_conf_c5_4.groupby(['predict_5_proba_4_bins', 'lab_perf_20d_class_5']).size().unstack(1)

bins = np.linspace(0.0995, 0.1035, 11)
df_conf_c10_9['predict_10_proba_9_bins'] = pd.cut(df_conf_c10_9['predict_10_proba_9'], bins=bins, include_lowest=True)
bar_data = df_conf_c10_9.groupby(['predict_10_proba_9_bins', 'lab_perf_20d_class_10']).size().unstack(1)


# Plot the grouped bar chart
bar_data.plot(kind='bar', figsize=(12, 6))
plt.xlabel('Probability Bins')
plt.ylabel('Count')
plt.title('Bar Chart of Predict_C_Proba_X by Lab_Perf_20d_Class_C')
plt.legend(title='lab_perf_20d_class')
plt.show()


In [7]:
# get stock info with get_info_all_stock

df_stock_infos=sio.get_info_all_stock(con_fwk )
print(f"{df_stock_infos.shape=}")
print(df_stock_infos.head())

df_stock_infos.shape=(403, 8)
   SK_SYMBOL     CODE                    NAME   TYPE  REGION CODE_YAHOO  \
0         12   2MX.PA             2MX ORGANIC  Stock  France     2MX.PA   
1         13   AAA.PA  ALAN ALLMAN ASSOCIATES  Stock  France     AAA.PA   
2         14  ABCA.PA           ABC ARBITRAGE  Stock  France    ABCA.PA   
3         15  ABEO.PA                    ABEO  Stock  France    ABEO.PA   
4         16  ABIO.PA                 ALBIOMA  Stock  France    ABIO.PA   

   TRADABLE SHARESOUTSTANDING  
0         1              None  
1         0          45675754  
2         1          59608879  
3         1           7543305  
4         1          32370737  


In [8]:
# Reset the index of df_all to make 'code' a regular column
df_all_reset = df_all.reset_index()

# Merge df_all with df_stock_infos on the 'code' column
df_all_merged = df_all_reset.merge(df_stock_infos[['CODE', 'SHARESOUTSTANDING', 'NAME', 'TRADABLE']], on='CODE', how='left')

# Restore the multi-index (datetime and code)
df_all_2 = df_all_merged.set_index(['OPEN_DATETIME', 'CODE'])

# Print the updated DataFrame
print(df_all_2.head())

                        OPEN    HIGH     LOW   CLOSE   VOLUME  pos_sma20  \
OPEN_DATETIME CODE                                                         
1991-10-23    BOI.PA  0.6832  0.6832  0.6832  0.6832    640.0     -0.030   
1991-10-24    BOI.PA  0.6635  0.6635  0.6635  0.6635   7840.0     -0.054   
1991-10-25    BOI.PA  0.6635  0.6635  0.6635  0.6635   7744.0     -0.050   
1991-10-28    BOI.PA  0.6543  0.6543  0.6543  0.6543  14848.0     -0.059   
1991-10-29    BOI.PA  0.6622  0.6622  0.6622  0.6622    800.0     -0.043   

                      pos_sma50  pos_sma200  pos_sma50_200  pos_sma20_50  ...  \
OPEN_DATETIME CODE                                                        ...   
1991-10-23    BOI.PA     -0.046       0.085          0.138        -0.016  ...   
1991-10-24    BOI.PA     -0.073       0.053          0.136        -0.019  ...   
1991-10-25    BOI.PA     -0.071       0.053          0.134        -0.022  ...   
1991-10-28    BOI.PA     -0.083       0.037          0.131    

In [9]:
#check type of f_all_2['CLOSE'] and df_all_2['SHARESOUTSTANDING']
print(f"{df_all_2['CLOSE'].dtype=} {df_all_2['SHARESOUTSTANDING'].dtype=}")

df_all_2['CLOSE'].dtype=dtype('float64') df_all_2['SHARESOUTSTANDING'].dtype=dtype('O')


In [10]:
from numpy import int64

df_all_2['NAME']=df_all_2['NAME'].astype(str)
df_all_2['TRADABLE']=df_all_2['TRADABLE'].astype(int)
df_all_2['SHARESOUTSTANDING']=df_all_2['SHARESOUTSTANDING'].fillna(0)
df_all_2['SHARESOUTSTANDING']=df_all_2['SHARESOUTSTANDING'].replace('',0)
df_all_2['SHARESOUTSTANDING']=df_all_2['SHARESOUTSTANDING'].astype(int64)
df_all_2['cap_M']=round((df_all_2['CLOSE']*df_all_2['SHARESOUTSTANDING'])/1000000.0,2)
df_all_2.head(5)

,,OPEN,HIGH,LOW,CLOSE,VOLUME,pos_sma20,pos_sma50,pos_sma200,pos_sma50_200,pos_sma20_50,...,lab_perf_50d_class_10,predict_10,predict_5,predict_10_proba,predict_5_proba,PART,SHARESOUTSTANDING,NAME,TRADABLE,cap_M
OPEN_DATETIME,CODE,,,,,,,,,,,,,,,,,,,,,
1991-10-23,BOI.PA,0.6832,0.6832,0.6832,0.6832,640.0,-0.030,-0.046,0.085,0.138,-0.016,...,3.0,6.0,1.0,"[0.09985782399798895, 0.09992670388165456, 0.1...","[0.19909589227087685, 0.20071967485772987, 0.1...",TRAIN,17545408,BOIRON,1,11.99
1991-10-24,BOI.PA,0.6635,0.6635,0.6635,0.6635,7840.0,-0.054,-0.073,0.053,0.136,-0.019,...,2.0,8.0,4.0,"[0.10004667817075666, 0.0998982506703846, 0.09...","[0.19938673862426753, 0.19926126592536392, 0.1...",TRAIN,17545408,BOIRON,1,11.64
1991-10-25,BOI.PA,0.6635,0.6635,0.6635,0.6635,7744.0,-0.050,-0.071,0.053,0.134,-0.022,...,3.0,6.0,4.0,"[0.10002173869508071, 0.09990738056821478, 0.0...","[0.19920730275897847, 0.19926594602352998, 0.1...",TRAIN,17545408,BOIRON,1,11.64
1991-10-28,BOI.PA,0.6543,0.6543,0.6543,0.6543,14848.0,-0.059,-0.083,0.037,0.131,-0.026,...,5.0,9.0,3.0,"[0.10002642307660348, 0.09988535349790527, 0.0...","[0.19942715141097156, 0.19882110780624856, 0.1...",TRAIN,17545408,BOIRON,1,11.48
1991-10-29,BOI.PA,0.6622,0.6622,0.6622,0.6622,800.0,-0.043,-0.071,0.049,0.129,-0.029,...,4.0,8.0,4.0,"[0.10000389870274415, 0.09989718304586509, 0.0...","[0.19942952202688463, 0.1995544997359649, 0.19...",TRAIN,17545408,BOIRON,1,11.62


In [12]:
print(df_all_2.shape)
joblib.dump(df_all_2, PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL2.pkl")

with open(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL_INFO2.txt", "w") as f:
    f.write(f"Columns:\n{list(df_all_2.columns)}\n")
    f.write(f"Index:\n{list(df_all_2.index.levels)}\n")


(671570, 136)


In [8]:
df_all_2=joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL2.pkl")
print(f"{df_all_2.shape=}")

df_all_2.shape=(671570, 136)


In [4]:
#  load BASE eand merge it with df_all_2
df_base = pd.read_csv(PATH_DATA_DTS+"PARIS_TREND_1D_20D_V3_BASE.zip",sep=",",index_col=["OPEN_DATETIME","CODE"],parse_dates=["OPEN_DATETIME"])
print(f"{df_base.shape=}")

df_base.shape=(1299247, 159)


In [9]:
# do a left join on df_base and df_all_2 on the index (OPEN_DATETIME, CODE)
# keep all the columns of df_base and df_all_2
df_all_3 = df_base.join(df_all_2, how='left', on=['OPEN_DATETIME','CODE'],  rsuffix='_val',)
print(f"{df_all_3.shape=}")

df_all_3.shape=(1299247, 295)


In [10]:
df_merged=df_all_3.copy()
df_merged=df_all_3[ ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME','pos_sma20_200','adx14_pos',
                    'lab_perf_20d_class_5', 'lab_perf_20d_class_10',
                    'predict_10', 'predict_5', 'predict_10_proba', 'predict_5_proba',
                      'PART', 'SHARESOUTSTANDING', 'NAME', 'TRADABLE', 'cap_M']]

print(df_merged.shape)

(1299247, 18)


In [16]:
# save df_merged in a file
joblib.dump(df_merged, PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL3.pkl")

['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL3.pkl']

In [11]:
df_merged=joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL3.pkl")
print(f"{df_merged.shape=}")

df_merged.shape=(1299247, 18)


In [5]:
# print min and max of df_merged.index.get_level_values('OPEN_DATETIME') group by df_merged['PART']
df_merged_reset = df_merged.reset_index()  # Reset the index to access 'OPEN_DATETIME' as a column
print(df_merged_reset.groupby('PART')['OPEN_DATETIME'].agg(['min', 'max']))
# print nb line where df_merged['PART']is null
print(f"Nb line where PART is null: {df_merged[df_merged['PART'].isnull()].shape[0]}")

             min        max
PART                       
CONF  2020-10-01 2023-11-01
TRAIN 1991-10-23 2017-07-31
VAL   2017-08-01 2020-09-30
Nb line where PART is null: 627677


In [12]:
# calculate values for 'PART', 'SHARESOUTSTANDING', 'NAME', 'TRADABLE', 'cap_M'
df_merged['PART'] = df_merged.index.get_level_values('OPEN_DATETIME').map(
    lambda x: 'TRAIN' if x < pd.Timestamp('2017-08-01') else 
              'VAL' if x < pd.Timestamp('2020-10-01') else 
              'CONF' if pd.notnull(x) else None
)
df_merged['NAME'] = df_merged.groupby(df_merged.index.get_level_values('CODE'))['NAME'].transform(lambda x: x.ffill().bfill())
df_merged['SHARESOUTSTANDING'] = df_merged.groupby(df_merged.index.get_level_values('CODE'))['SHARESOUTSTANDING'].transform(lambda x: x.ffill().bfill())
df_merged['TRADABLE'] = df_merged.groupby(df_merged.index.get_level_values('CODE'))['TRADABLE'].transform(lambda x: x.ffill().bfill())
df_merged['cap_M'] = round((df_merged['CLOSE']*df_merged['SHARESOUTSTANDING'])/1000000.0,2)
# drop line where NAME is null
# df_merged.dropna(subset=['NAME'], inplace=True)

In [15]:
print(f"Nb line where PART is null: {df_merged[df_merged['PART'].isnull()].shape[0]}")
print(f"Nb line where NAME is null: {df_merged[df_merged['NAME'].isnull()].shape[0]}")
print(f"Nb line where SHARESOUTSTANDING is null: {df_merged[df_merged['SHARESOUTSTANDING'].isnull()].shape[0]}")
print(f"Nb line where TRADABLE is null: {df_merged[df_merged['TRADABLE'].isnull()].shape[0]}")
print(f"Nb line where cap_M is null: {df_merged[df_merged['cap_M'].isnull()].shape[0]}")
print(f"{df_merged.shape=}")

df_merged.dropna(subset=['NAME'], inplace=True)
print(f"{df_merged.shape=}")

Nb line where PART is null: 0
Nb line where NAME is null: 65455
Nb line where SHARESOUTSTANDING is null: 65455
Nb line where TRADABLE is null: 65455
Nb line where cap_M is null: 65455
df_merged.shape=(1299247, 18)
df_merged.shape=(1233792, 18)


In [16]:
# print for the code AI.PA the different NAME of the dataset 
print(df_merged[df_merged.index.get_level_values('CODE')=='AI.PA']['NAME'].unique())

['AIR LIQUIDE']


In [18]:
joblib.dump(df_merged, PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL4.pkl")

['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL4.pkl']

In [17]:
# Extract the datetime level from the MultiIndex and apply strftime
df_ai = df_merged[df_merged.index.get_level_values('CODE') == 'AI.PA']
df_ai['YYYYMM'] = df_ai.index.get_level_values('OPEN_DATETIME').strftime('%Y%m')
# Count rows by YYYYMM and sort by YYYYMM
df_ai_count = df_ai.groupby('YYYYMM').size().reset_index(name='count')
# print df_ai for year 2022
print(df_ai_count[df_ai_count['YYYYMM'].str.startswith('2022')])

     YYYYMM  count
264  202201     21
265  202202     20
266  202203     23
267  202204     19
268  202205     22
269  202206     22
270  202207     21
271  202208     23
272  202209     22
273  202210     21
274  202211     22
275  202212     21


C:\Users\Benoit\AppData\Local\Temp\ipykernel_12496\2564425692.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ai['YYYYMM'] = df_ai.index.get_level_values('OPEN_DATETIME').strftime('%Y%m')


Prep for model not adjusted

In [4]:
# load PARIS_TREND_1D_20D_V4_filtered.zip
df_filtered = pd.read_csv(PATH_DATA_DTS+"PARIS_TREND_1D_20D_V4_filtered.zip",sep=",",index_col=["OPEN_DATETIME","CODE"],parse_dates=["OPEN_DATETIME"])
print(f"{df_filtered.shape=}")

df_filtered.shape=(716338, 117)


In [ ]:
# save in a file the liste of code  in the format 'CODE_1','CODE_2',...,'CODE_N'
list_code = df_filtered.index.get_level_values('CODE').unique().tolist()
print(f"{list_code=}")
list_code_str = ','.join([f"'{code}'" for code in list_code])
print(f"{list_code_str=}")
# save the list_code_str in a file
with open(PATH_DATA_DTS+"PARIS_TREND_1D_20D_V4_filtered_list_code.txt", "w") as f:
    f.write(f"{list_code_str}\n")


list_code=['BOI.PA', 'ELEC.PA', 'CDI.PA', 'TEP.PA', 'GFC.PA', 'LI.PA', 'RE.PA', 'SAVE.PA', 'VIRP.PA', 'LOUP.PA', 'BN.PA', 'TFI.PA', 'CRI.PA', 'MF.PA', 'ERF.PA', 'BEN.PA', 'TRI.PA', 'SOI.PA', 'GLO.PA', 'PIG.PA', 'ALDLT.PA', 'ALGIL.PA', 'AM.PA', 'BIG.PA', 'BNP.PA', 'CAT31.PA', 'CDA.PA', 'CEN.PA', 'CMO.PA', 'CRAP.PA', 'CRLO.PA', 'CRSU.PA', 'DIM.PA', 'DSY.PA', 'FGR.PA', 'IMDA.PA', 'LSS.PA', 'PERR.PA', 'RAL.PA', 'RCO.PA', 'RF.PA', 'RI.PA', 'SBT.PA', 'SDG.PA', 'TFF.PA', 'THEP.PA', 'VK.PA', 'GLE.PA', 'BUR.PA', 'AC.PA', 'BLC.PA', 'BON.PA', 'CA.PA', 'CGG.PA', 'CIV.PA', 'CRTO.PA', 'ES.PA', 'FII.PA', 'HO.PA', 'PVL.PA', 'RNO.PA', 'SAN.PA', 'SCR.PA', 'SU.PA', 'SCHP.PA', 'VCT.PA', 'ASY.PA', 'ENGI.PA', 'CRBP2.PA', 'MRN.PA', 'SGO.PA', 'AI.PA', 'EL.PA', 'ALFLE.PA', 'ML.PA', 'EXE.PA', 'FR.PA', 'PEUG.PA', 'BAIN.PA', 'KER.PA', 'KOF.PA', 'LPE.PA', 'VRAP.PA', 'DKUPL.PA', 'RUI.PA', 'STF.PA', 'ATE.PA', 'LACR.PA', 'CAP.PA', 'ATO.PA', 'PARP.PA', 'ERA.PA', 'LIN.PA', 'QDT.PA', 'VIE.PA', 'VIL.PA', 'NK.PA', 'SOP.PA